In [1]:
#=========================================================
# merge_instruments.py
# Author: McKenna W. Stanford
# Pulls in various datastreams and merges them into a single
# dictinary for a given date. For time and/or height data,
# the data is interpolated onto the BASTA time-height grid
# of 12 seconds and 25 m. Writes the resulting dictionary
# to a pickle file.
#=========================================================

In [2]:
#--------------------------------
# Imports
#--------------------------------
import numpy as np
import matplotlib.pyplot as plt
import glob
import xarray
import datetime
import calendar
from matplotlib.gridspec import GridSpec
import matplotlib.dates as mdates
import matplotlib
import pickle
import pandas as pd
import os
from file_struct import file_struct as fs
from load_sonde_data import load_sonde_data
from give_me_files_and_subfolders import give_me_files_and_subfolders
from scipy import ndimage
from scipy.ndimage import gaussian_filter
from scipy.interpolate import NearestNDInterpolator as nn
from scipy.interpolate import griddata as griddata
from calculate_theta_and_more import calculate_theta_and_more
import pandas
import metpy.calc as mpcalc
from metpy.units import units
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D
from scipy.interpolate import interp2d
from scipy.interpolate import griddata
#import act
import metpy
import metpy.calc as mpcalc
from metpy.cbook import get_test_data
from metpy.plots import add_metpy_logo, SkewT
from metpy.units import units
import time
from dask.distributed import Client, progress, LocalCluster
import csv
#import seaborn as sns
#--------------------------------------------
#--------------------------------------------

In [3]:
#--------------------------------------------
# Functions
#--------------------------------------------
def toTimestamp(d):
    return calendar.timegm(d.timetuple())

def toDatetime(d):
    return datetime.datetime.utcfromtimestamp(d)

def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return array[idx],idx   

# function to make serial date numbers which are the number of days that have passed
# since epoch beginning given as days.fraction_of_day
def datenum(d):
        return 366 + d.toordinal() + (d - datetime.datetime.fromordinal(d.toordinal())).total_seconds()/(24*60*60)
#--------------------------------------------
#--------------------------------------------

In [4]:
#==================================================
# Grab BASTA files
#==================================================
basta_path = '/mnt/raid/mwstanfo/micre_data/micre_basta/BASTA_25m/'
basta_files = glob.glob(basta_path+'*.nc')
basta_files = sorted(basta_files)
basta_files = np.array(basta_files)

basta_dates_dt = []
for ii in range(len(basta_files)):
    fname = basta_files[ii]
    tmp_str = fname.split('_')
    tmp_str = tmp_str[-1]
    tmp_str = tmp_str.split('.')
    tmp_str = tmp_str[0]
    tmp_year = int(tmp_str[0:4])
    tmp_month = int(tmp_str[4:6])
    tmp_day = int(tmp_str[6:8])
    basta_dates_dt.append(datetime.datetime(tmp_year,tmp_month,tmp_day,0,0,0))
basta_dates_dt = np.array(basta_dates_dt)
dates = basta_dates_dt.copy()

In [5]:
#==================================================
# Grab ARM Ceilometer files
#==================================================
arm_ceil_path = '/mnt/raid/mwstanfo/micre_data/micre_ceil/'
arm_ceil_files = glob.glob(arm_ceil_path+'*.nc')
arm_ceil_files = sorted(arm_ceil_files)
arm_ceil_files = np.array(arm_ceil_files)

arm_ceil_dates_dt = []
for ii in range(len(arm_ceil_files)):
    fname = arm_ceil_files[ii]
    tmp_str = fname.split('/')
    tmp_str = tmp_str[-1]
    tmp_str = tmp_str.split('.')
    tmp_str = tmp_str[2]
    tmp_year = int(tmp_str[0:4])
    tmp_month = int(tmp_str[4:6])
    tmp_day = int(tmp_str[6:8])
    arm_ceil_dates_dt.append(datetime.datetime(tmp_year,tmp_month,tmp_day,0,0,0))
arm_ceil_dates_dt = np.array(arm_ceil_dates_dt)

# Limit ceilometer files to encompass only BASTA dates
tmpid = np.where((arm_ceil_dates_dt >= basta_dates_dt[0]) & (arm_ceil_dates_dt <= basta_dates_dt[-1]))[0]
arm_ceil_files = arm_ceil_files[tmpid]
arm_ceil_dates_dt = arm_ceil_dates_dt[tmpid]

In [6]:
#==================================================
# Grab University of Canterbury Ceilometer files
#==================================================
aad_ceil_path = '/mnt/raid/mwstanfo/micre_data/aad_ceil/merged/'
aad_ceil_files = glob.glob(aad_ceil_path+'*.nc')
aad_ceil_files = sorted(aad_ceil_files)
aad_ceil_files = np.array(aad_ceil_files)


aad_ceil_dates_dt = []
for ii in range(len(aad_ceil_files)):
    fname = aad_ceil_files[ii]
    tmp_str = fname.split('/')
    tmp_str = tmp_str[-1]
    tmp_str = tmp_str.split('.')
    tmp_str = tmp_str[0]
    tmp_str = tmp_str.split('_')
    tmp_year = int(tmp_str[3])
    tmp_month = int(tmp_str[4])
    tmp_day = int(tmp_str[5])
    aad_ceil_dates_dt.append(datetime.datetime(tmp_year,tmp_month,tmp_day))
    
aad_ceil_dates_dt = np.array(aad_ceil_dates_dt)
# Limit ceilometer files to encompass only BASTA dates
tmpid = np.where((aad_ceil_dates_dt >= basta_dates_dt[0]) & (aad_ceil_dates_dt <= basta_dates_dt[-1]))[0]
aad_ceil_dates_dt = aad_ceil_dates_dt[tmpid] 
aad_ceil_files = aad_ceil_files[tmpid]

In [7]:
#==================================================
# Grab surface meteorology files
#==================================================
sfc_path = '/mnt/raid/mwstanfo/micre_data/micre_sfc/'
sfc_files = glob.glob(sfc_path+'*.nc')
sfc_files = sorted(sfc_files)
sfc_files = np.array(sfc_files)

sfc_dates_dt = []
for ii in range(len(sfc_files)):
    fname = sfc_files[ii]
    tmp_str = fname.split('/')
    tmp_str = tmp_str[-1]
    tmp_str = tmp_str.split('.')
    tmp_str = tmp_str[2]
    tmp_year = int(tmp_str[0:4])
    tmp_month = int(tmp_str[4:6])
    tmp_day = int(tmp_str[6:8])
    sfc_dates_dt.append(datetime.datetime(tmp_year,tmp_month,tmp_day,0,0,0))
sfc_dates_dt = np.array(sfc_dates_dt)

# Limit sfc met files to encompass only BASTA dates
tmpid = np.where((sfc_dates_dt >= basta_dates_dt[0]) & (sfc_dates_dt <= basta_dates_dt[-1]))[0]
sfc_dates_dt = sfc_dates_dt[tmpid] 
sfc_files = sfc_files[tmpid]

In [8]:
#==================================================
# Grab satellite files
#==================================================
sat_path = '/mnt/raid/mwstanfo/micre_data/visst_gridded/'
sat_files = glob.glob(sat_path+'*.nc')
sat_files = sorted(sat_files)
sat_files = np.array(sat_files)

sat_dates_dt = []
for ii in range(len(sat_files)):
    fname = sat_files[ii]
    tmp_str = fname.split('/')
    tmp_str = tmp_str[-1]
    tmp_str = tmp_str.split('.')
    tmp_str = tmp_str[0]
    tmp_str = tmp_str.split('_')
    tmp_str = tmp_str[2:]
    tmp_year = int(tmp_str[0])
    tmp_month = int(tmp_str[1])
    tmp_day = int(tmp_str[2])
    sat_dates_dt.append(datetime.datetime(tmp_year,tmp_month,tmp_day,0,0,0))
sat_dates_dt = np.array(sat_dates_dt)

# sort files according to dates
sort_id = np.argsort(sat_dates_dt)
sat_dates_dt = sat_dates_dt[sort_id]
sat_files = sat_files[sort_id]

# Limit sat files to encompass only BASTA dates
tmpid = np.where((sat_dates_dt >= basta_dates_dt[0]) & (sat_dates_dt <= basta_dates_dt[-1]))[0]
sat_dates_dt = sat_dates_dt[tmpid] 
sat_files = sat_files[tmpid]
#--------------------------------------------

In [9]:
#==================================================
# Grab optics files
#==================================================
path = '/mnt/raid/mwstanfo/micre_data/micre_optics/'
optics_files = glob.glob(path+'*.cdf')
optics_files = sorted(optics_files)
optics_files = np.array(optics_files)
nf = len(optics_files)
#print(optics_files)

optics_dates_dt = []
for ff in range(nf):
    optics_file = optics_files[ff]
    optics_file = str.split(optics_file,'/')[-1]
    optics_file = str.split(optics_file,'.')[-3]
    year = int(optics_file[0:4])
    month = int(optics_file[4:6])
    day = int(optics_file[6:8])
    optics_date_dt = datetime.datetime(year,month,day)
    optics_dates_dt.append(optics_date_dt)
optics_dates_dt = np.array(optics_dates_dt)
# limit to only basta times
tmpid = np.where((optics_dates_dt >= basta_dates_dt[0]) & (optics_dates_dt <= basta_dates_dt[-1]))[0]
optics_files = optics_files[tmpid]
optics_dates_dt = optics_dates_dt[tmpid]

In [10]:
if False:
#if True:
    sns.set_theme()
    #sns.set_style('dark')
    sns.set_style('ticks')
    sns.set(rc={'axes.facecolor':'lavender','axes.edgecolor': 'black','grid.color':'white'})
    sns.set_context('talk') 

    # Look at optics files
    ii=0
    for file in optics_files[10:]:
        print(file)
        ncfile = xarray.open_dataset(file,decode_times=False)
        base_time = ncfile['base_time'].values
        time_offset = ncfile['time_offset'].values
        qc_time = ncfile['qc_time'].values
        tau_inst = ncfile['optical_depth_instantaneous'].values
        qc_tau_inst = ncfile['qc_optical_depth_instantaneous'].values
        reff_inst = ncfile['effective_radius_instantaneous'].values
        qc_reff_inst = ncfile['qc_effective_radius_instantaneous'].values
        tau_avg = ncfile['optical_depth_instantaneous'].values
        qc_tau_avg = ncfile['qc_optical_depth_average'].values
        reff_avg = ncfile['effective_radius_average'].values
        qc_reff_avg = ncfile['qc_effective_radius_average'].values
        trans1 = ncfile['total_transmittance_filter1'].values
        trans2 = ncfile['total_transmittance_filter2'].values
        trans3 = ncfile['total_transmittance_filter3'].values
        trans4 = ncfile['total_transmittance_filter4'].values
        trans5 = ncfile['total_transmittance_filter5'].values
        lwp = ncfile['lwp'].values
        qc_lwp = ncfile['qc_lwp'].values
        pwv = ncfile['pwv'].values
        qc_pwv = ncfile['qc_pwv'].values
        cf = ncfile['cloudfraction'].values
        qc_cf = ncfile['qc_cloudfraction'].values
        ncfile.close()
        time_ts = base_time + time_offset
        time_dt = np.array([toDatetime(time_ts[dd]) for dd in range(len(time_ts))])

              

        fig = plt.figure(figsize=(20,14))
        ax_tau = fig.add_subplot(321)
        ax_reff = fig.add_subplot(322)
        ax_lwp = fig.add_subplot(323)
        ax_cf = fig.add_subplot(324)
        ax_trans = fig.add_subplot(325)
        Fontsize=16
        dfmt = mdates.DateFormatter('%H:%M')

        ax_list = [ax_tau,ax_reff,ax_lwp,ax_cf,ax_trans]
        for ax in ax_list:
            ax.grid(True)
            ax.tick_params(labelsize=Fontsize)
            ax.xaxis.set_major_formatter(dfmt)
            ax.set_xlabel('UTC Time [HH:MM]',fontsize=Fontsize)
            ax.set_axisbelow(True)

        ax_tau.set_title('Optical Depth ($\\tau$)',fontsize=Fontsize*1.5,fontweight='bold')
        ax_tau.set_ylabel('$\\tau$',fontsize=Fontsize)
        ax_reff.set_title('Effective Radius ($R_{eff}$)',fontsize=Fontsize*1.5,fontweight='bold')
        ax_reff.set_ylabel('$R_{eff}$ [$\\mu$m]',fontsize=Fontsize)
        ax_lwp.set_title('Liquid Water Path (LWP)',fontsize=Fontsize*1.5,fontweight='bold')
        ax_lwp.set_ylabel('LWP [g m$^{-2}$]',fontsize=Fontsize)
        ax_cf.set_title('Cloud Fraction',fontsize=Fontsize*1.5,fontweight='bold')
        ax_cf.set_ylabel('Cloud Fraction',fontsize=Fontsize)
        ax_tau.set_yscale('log')
        ax_lwp.set_yscale('log')
        ax_trans.set_title('Total transmittance of Narrowband\nHemispheric Irradiance',fontsize=Fontsize,fontweight='bold')
        ax_trans.set_ylabel('Transmittance',fontsize=Fontsize)
        ax_cf.set_ylabel('Cloud Fraction',fontsize=Fontsize)
        
        # Plot
        ax_tau.plot(time_dt,tau_inst,label='$\\tau_{inst}$',c='black',lw=2)
        ax_tau.plot(time_dt,tau_avg,label='$\\tau_{avg}$',c='red',lw=2,ls='dashed')
        ax_tau.legend(loc='upper left',fontsize=Fontsize)
        ax_reff.plot(time_dt,reff_inst,label='$R_{eff,inst}$',c='black',lw=2)
        ax_reff.plot(time_dt,reff_avg,label='$R_{eff,avg}$',c='red',lw=2,ls='dashed')
        ax_reff.legend(loc='upper left',fontsize=Fontsize)
        ax_lwp.plot(time_dt,lwp,c='black',lw=2)
        ax_cf.plot(time_dt,cf,c='black',lw=2)
        ax_trans.plot(time_dt,trans1,c='black',lw=1,label='Filter 1')
        ax_trans.plot(time_dt,trans1,c='red',lw=1,label='Filter 2')
        ax_trans.plot(time_dt,trans3,c='darkorange',lw=1,label='Filter 3')
        ax_trans.plot(time_dt,trans4,c='blue',lw=1,label='Filter 4')
        ax_trans.plot(time_dt,trans5,c='deepskyblue',lw=1,label='Filter 5')
        ax_trans.legend(loc='upper center',fontsize=Fontsize,ncol=2)

        dumdate = datetime.datetime(time_dt[0].year,time_dt[0].month,time_dt[0].day)
        dumdate = dumdate.strftime('%Y/%m/%d')
        plt.suptitle(dumdate,fontsize=Fontsize*2.,fontweight='bold')
        plt.subplots_adjust(hspace=0.4)

        plt.show()
        print(aaaa)

        dumdate = datetime.datetime(time_dt[0].year,time_dt[0].month,time_dt[0].day)
        dumdate = dumdate.strftime('%Y%m%d')
        fig_path = '/home/mwstanfo/figures/micre_v2/optics/'
        outfile = 'optics_{}.png'.format(dumdate)
        plt.savefig(fig_path+outfile,dpi=200,bbox_inches='tight')       
        plt.close()    
        ii+=1

        #print(aaa)

In [11]:
#==================================================
# Grab sounding files
#==================================================
path = '/mnt/raid/mwstanfo/micre_data/micre_soundings/'
sonde_files = glob.glob(path+'*.nc')
sonde_files = sorted(sonde_files)
sonde_files = np.array(sonde_files)
nf = len(sonde_files)

sonde_dates_dt = []
sonde_times_dt = []
for ff in range(nf):
    sonde_file = sonde_files[ff]
    sonde_file = str.split(sonde_file,'/')[-1]
    sonde_file = str.split(sonde_file,'.')[0]
    sonde_file_str1 = str.split(sonde_file,'_')[0]
    sonde_file_str2 = str.split(sonde_file,'_')[1]
    year = int(sonde_file_str1[0:4])
    month = int(sonde_file_str1[4:6])
    day = int(sonde_file_str1[6:8])
    hour = int(sonde_file_str2[0:2])
    minute = int(sonde_file_str2[2:4])
    sonde_time_dt = datetime.datetime(year,month,day,hour,minute)
    sonde_date_dt = datetime.datetime(year,month,day)
    sonde_dates_dt.append(sonde_date_dt)
    sonde_times_dt.append(sonde_time_dt)
sonde_dates_dt = np.array(sonde_dates_dt)
sonde_times_dt = np.array(sonde_times_dt)

In [12]:
#==================================================
# Grab cluster files
#==================================================
cluster_file ='/mnt/raid/mwstanfo/micre/cluster_information_all.xlsx'

clusters = pd.read_excel(cluster_file)
clusters = np.array(clusters)
cluster_number = clusters[:,0]
cluster_dates = clusters[:,1]
cluster_ids = clusters[:,2]

cluster_dates_dt = []
cluster_times_dt = []
for cluster_date in cluster_dates:
    tmpstr1,tmpstr2 = str.split(cluster_date,'_')
    tmpyear = int(tmpstr1[0:4])
    tmpmonth = int(tmpstr1[4:6])
    tmpday = int(tmpstr1[6:8])
    tmphour = int(tmpstr2[0:2])
    tmpminute = int(tmpstr2[2:4])
    cluster_times_dt.append(datetime.datetime(tmpyear,tmpmonth,tmpday,tmphour,tmpminute))
    cluster_dates_dt.append(datetime.datetime(tmpyear,tmpmonth,tmpday))
cluster_dates_dt = np.array(cluster_dates_dt)
cluster_times_dt = np.array(cluster_times_dt)

In [13]:
#==================================================
# Grab disdrometer (PIRAT) files
#==================================================
dis_path = '/mnt/raid/mwstanfo/micre_data/micre_disdrometer/PIRAT_rates_csv/'
dis_files = glob.glob(dis_path+'*.csv')
dis_files = sorted(dis_files)
dis_files = np.array(dis_files)
dis_dates_dt = []

for ii in range(len(dis_files)):
    tmp_str = dis_files[ii].split('/')
    tmp_str = tmp_str[-1]
    tmp_str = tmp_str.split('_')
    tmp_str = tmp_str[1]
    tmp_str = tmp_str.split('.')
    tmp_str = tmp_str[0]
    tmp_year = int(tmp_str[0:4])
    tmp_month = int(tmp_str[4:6])
    tmp_day = int(tmp_str[6:8])
    dis_dates_dt.append(datetime.datetime(tmp_year,tmp_month,tmp_day))
    #dis_data = pd.read_csv(dis_files[ii],delimiter=',',header=0)
    #dis_data = np.array(dis_data)
    #dis_time = dis_data[:,0] # time in fraction of UTC hour
    #dis_DQ = dis_data[:,1] # 0: good, != 0: bad
    #dis_R = dis_data[:,2] # precipitation rate in mm/hr
dis_dates_dt = np.array(dis_dates_dt)

# Limit disdrometer files to encompass only BASTA dates
tmpid = np.where((dis_dates_dt >= basta_dates_dt[0]) & (dis_dates_dt <= basta_dates_dt[-1]))[0]
dis_dates_dt = dis_dates_dt[tmpid]
dis_files = dis_files[tmpid]

In [14]:
#path = '/mnt/raid/mwstanfo/micre/merged_instrument_files/'
#files = glob.glob(path+'*.p')
#files = sorted(files)
#infile = files[0]
#merged_dict = pickle.load(open(infile,"rb"))

In [15]:
def process_dis(date,dis_dates_dt,dis_files,avg_bool,basta_time_ts):
    tmpid = np.where(dis_dates_dt == date)
    if np.size(tmpid) == 0.:
        dis_present_flag = False
        dis_dict = None
        return dis_present_flag,dis_dict
    elif np.size(tmpid) > 0.:
        dis_present_flag = True
        tmpid = tmpid[0][0]
        current_dis_file = dis_files[tmpid]
        current_dis_date = dis_dates_dt[tmpid]

        dis_data = pd.read_csv(current_dis_file,delimiter=',',header=0)
        dis_data = np.array(dis_data)
        dis_time = dis_data[:,0] # time in fraction of UTC hour
        dis_dq = dis_data[:,1] # 0: good, != 0: bad
        dis_precip_rate = dis_data[:,2] # precipitation rate in mm/hr

        # convert dis_time from fraction of UTC hour to datetime object and timestamp
        dis_date = current_dis_date
        dis_year = dis_date.year
        dis_month = dis_date.month
        dis_day = dis_date.day
        dis_hour = np.array([np.floor(dis_time[dd]) for dd in range(len(dis_time))])
        fraction_of_hour = np.array([(dis_time[dd]-dis_hour[dd]) for dd in range(len(dis_time))])
        dis_minute = fraction_of_hour*60.

        dis_time_dt = np.array([datetime.datetime(dis_year,\
                                                  dis_month,\
                                                  dis_day,\
                                                  int(dis_hour[dd]),\
                                                  int(dis_minute[dd])) for dd in range(len(dis_time))])
        dis_time_ts = np.array([toTimestamp(dis_time_dt[dd]) for dd in range(len(dis_time))])

        # NaN out bad values
        dis_precip_rate[dis_dq > 0.] = np.nan
        
        dis_dict = {'precip_rate':dis_precip_rate,'time_dt':dis_time_dt,'time_ts':dis_time_ts} 
        return dis_present_flag, dis_dict

In [16]:
def process_sfc_met(date,sfc_dates_dt,sfc_files,avg_bool,basta_time_ts):

    tmpid = np.where(sfc_dates_dt == date)
    if np.size(tmpid) == 0.:
        sfc_met_present_flag = False
        sfc_met_out_dict = None
        return sfc_met_present_flag, sfc_met_out_dict
    elif np.size(tmpid) > 0.:        
        sfc_met_present_flag = True
        tmpid = tmpid[0][0]
        current_sfc_file = sfc_files[tmpid]
        
        ncfile = xarray.open_dataset(current_sfc_file,decode_times=False)
        sfc_dims = ncfile.dims
        sfc_base_time = np.array(ncfile['base_time'].copy())
        sfc_num_times = sfc_dims['time']
        sfc_time_offset = np.array(ncfile['time_offset'].copy())
        sfc_temperature = np.array(ncfile['temperature'].copy())
        sfc_pressure = np.array(ncfile['pressure'].copy())
        sfc_rh = np.array(ncfile['relative_humidity'].copy())
        sfc_wind_speed = np.array(ncfile['wind_speed'].copy())
        sfc_wind_dir = np.array(ncfile['wind_direction'].copy())
        sfc_precip = np.array(ncfile['cumulative_precipitation'].copy())
        ncfile.close()

        sfc_time_ts = [int(sfc_base_time + sfc_time_offset[dd]) for dd in range(sfc_num_times)]
        sfc_time_dt = [toDatetime(sfc_time_ts[dd]) for dd in range(sfc_num_times)]

        # NaN out missing values
        # original missing value is -9999.
        sfc_temperature[sfc_temperature < -950.] = np.nan
        sfc_pressure[sfc_pressure < -950.] = np.nan
        sfc_rh[sfc_rh < -950.] = np.nan
        sfc_wind_speed[sfc_wind_speed < -950.] = np.nan
        sfc_wind_dir[sfc_wind_dir < -950.] = np.nan
        sfc_precip[sfc_precip < -950.] = np.nan
        
        # interpolate sfc variables to basta time grid
        sfc_temperature_interp = np.interp(basta_time_ts,sfc_time_ts,sfc_temperature)
        sfc_rh_interp = np.interp(basta_time_ts,sfc_time_ts,sfc_rh)
        sfc_wind_speed_interp = np.interp(basta_time_ts,sfc_time_ts,sfc_wind_speed)
        sfc_wind_dir_interp = np.interp(basta_time_ts,sfc_time_ts,sfc_wind_dir,period=360)
        sfc_pressure_interp = np.interp(basta_time_ts,sfc_time_ts,sfc_pressure)


        sfc_met_out_dict = {'native_temperature':sfc_temperature,\
                    'native_rh':sfc_rh,\
                    'native_pressure':sfc_pressure,\
                    'native_wind_speed':sfc_wind_speed,\
                    'native_wind_dir':sfc_wind_dir,\
                    'native_precip':sfc_precip,\
                    'native_time_dt':sfc_time_dt,\
                    'native_time_ts':sfc_time_ts,\
                    'temperature':sfc_temperature_interp,\
                    'rh':sfc_rh_interp,\
                    'wind_speed':sfc_wind_speed_interp,\
                    'wind_dir':sfc_wind_dir_interp,\
                    'pressure':sfc_pressure_interp,\
                   }

        return sfc_met_present_flag,sfc_met_out_dict
    

In [17]:
def process_sat(date,sat_dates_dt,sat_files):

    tmpid = np.where(sat_dates_dt == date)
    if np.size(tmpid) == 0.:
        sat_present_flag = False
        sat_out_dict = None
        return sat_present_flag,sat_out_dict
    elif np.size(tmpid) > 0.:
        sat_present_flag = True
        tmpid = tmpid[0][0]
        current_sat_file = sat_files[tmpid]

        ncfile = xarray.open_dataset(current_sat_file,decode_times=False)
        sat_time_epoch = np.array(ncfile['time_epoch'].copy())
        sat_lat = np.array(ncfile['lat'].copy())
        sat_lon = np.array(ncfile['lon'].copy())
        sat_visible_reflectance = np.array(ncfile['visible_reflectance'].copy())
        sat_ir_brightness_temperature = np.array(ncfile['ir_brightness_temperature'].copy())
        sat_effective_temperature = np.array(ncfile['effective_temperature'].copy())
        sat_lwp = np.array(ncfile['lwp'].copy())
        sat_iwp = np.array(ncfile['iwp'].copy())
        sat_ice_re = np.array(ncfile['ice_re'].copy())
        sat_ice_de = np.array(ncfile['ice_de'].copy())
        sat_liq_re = np.array(ncfile['liq_re'].copy())
        sat_optical_depth = np.array(ncfile['optical_depth'].copy())
        ncfile.close()
        sat_time_ts = sat_time_epoch.copy()
        sat_time_dt = np.array([toDatetime(sat_time_ts[dd]) for dd in range(len(sat_time_ts))])
        

        sat_out_dict = {'time_ts':sat_time_ts,\
                        'time_dt':sat_time_dt,\
                        'lon':sat_lon,\
                        'lat':sat_lat,\
                        'visible_reflectance':sat_visible_reflectance,\
                        'ir_brightness_temperature':sat_ir_brightness_temperature,\
                        'ir_effective_temperature':sat_effective_temperature,\
                        'lwp':sat_lwp,\
                        'iwp':sat_iwp,\
                        'ice_re':sat_ice_re,\
                        'ice_de':sat_ice_de,\
                        'liq_re':sat_liq_re,\
                        'optical_depth':sat_optical_depth,\
                       }
        

        return sat_present_flag, sat_out_dict

In [18]:
#target_date = datetime.datetime(2016,5,23)
#dumid = np.where(basta_dates_dt == target_date)
#in_date = basta_dates_dt[dumid[0]]
#interp_sonde_present_flag, sonde_out_dict = interp_sondes(in_date,sonde_dates_dt,sonde_times_dt,sonde_files,merged_dict['basta']['time_dt'],merged_dict['basta']['height'])
#print('done')

In [19]:
def process_native_sondes(date,sonde_dates_dt,sonde_times_dt,sonde_files,cluster_ids,cluster_times_dt):
    sonde_id = np.where(sonde_dates_dt == date)[0]

    # Pull out 3 sounding times (if available) to be included in the output file.
    num_soundings = len(sonde_times_dt)
    date = date[0]
    target_time_1 = datetime.datetime(date.year,date.month,date.day,0)
    target_time_2 = datetime.datetime(date.year,date.month,date.day,12)
    tmp_time_delta = datetime.timedelta(days=1)
    target_time_3 = datetime.datetime(date.year,date.month,date.day,0) + tmp_time_delta
    sonde_times_dt = np.array(sonde_times_dt)

    
    
    tmpid = np.where((sonde_times_dt > (target_time_1 - datetime.timedelta(hours=2))) & (sonde_times_dt < (target_time_1 + datetime.timedelta(hours=2))))
    if np.size(tmpid) > 0:
        sonde_1_id = tmpid[0][0]
        sonde_1_flag = True
    else:
        sonde_1_flag = False
        sonde_1_id = -999.
    tmpid = np.where((sonde_times_dt > (target_time_2 - datetime.timedelta(hours=2))) & (sonde_times_dt < (target_time_2 + datetime.timedelta(hours=2))))
    if np.size(tmpid) > 0:
        sonde_2_id = tmpid[0][0]
        sonde_2_flag = True
    else:
        sonde_2_flag = False
        sonde_2_id = -999.
    tmpid = np.where((sonde_times_dt > (target_time_3 - datetime.timedelta(hours=2))) & (sonde_times_dt < (target_time_3 + datetime.timedelta(hours=2))))
    if np.size(tmpid) > 0:
        sonde_3_id = tmpid[0][0]
        sonde_3_flag = True
    else:
        sonde_3_flag = False 
        sonde_3_id = -999.
    
    sonde_flags = [sonde_1_flag,sonde_2_flag,sonde_3_flag]
    sonde_ids = [sonde_1_id,sonde_2_id,sonde_3_id]    



    native_sonde_dict = {}
    for jj in range(len(sonde_flags)):
        if not sonde_flags[jj]:
            native_sonde_dict[str(int(jj+1))] = None
            continue
        elif sonde_flags[jj]:
            current_sonde_id = sonde_ids[jj]
            current_sonde_file = sonde_files[current_sonde_id]
            current_sonde_time = sonde_times_dt[current_sonde_id]
            dumid = np.where(cluster_times_dt == current_sonde_time)
            if np.size(dumid) > 0.:
                current_cluster_id = cluster_ids[dumid[0][0]]
            else:
                current_cluster_id = None
            # Calculate everything
            current_sonde_file = current_sonde_file.split('/')[-1]
            path = '/mnt/raid/mwstanfo/micre_data/micre_soundings/'
            file_size = os.stat(path+current_sonde_file).st_size/1.e3
            fstruct = fs(current_sonde_file,path,file_size)
            Sondetmp = load_sonde_data(fstruct)
            
            #['site_lat', 'site_lon', 'sfc_pressure', 'sfc_temp', 'sfc_humidity',\
            # 'sfc_wind_speed', 'sfc_wind_direction', 'dewpoint_temp', 'drybulb_temp', \
            # 'RH', 'pressure', 'wind_direction', 'u_wind', 'v_wind', 'wind_speed', \
            # 'ascent_rate', 'lat', 'lon', 'alt', 'time', 'units', 'long_name']            

            max_alt = np.max(Sondetmp['alt'])
            if max_alt < 10.:
                print('Sonde failed to reach 10 km. Therefore omitting this sounding.')
                native_sonde_dict[str(int(jj+1))] = None
                sonde_flags[jj] = False
                continue
            else:
                pass
            
            # Calculate EIS, LTS, and LCL
            Moretmp = calculate_theta_and_more(Sondetmp['drybulb_temp'],Sondetmp['pressure'],\
                                           RH=Sondetmp['RH'],use_T_K=True,\
                                          sat_pres_formula='Emmanuel')
            
            nearest_val_700hpa,nearest_id_700hpa = find_nearest(Sondetmp['pressure'],700)
            theta_700hpa = Moretmp['Theta'][nearest_id_700hpa] # in K
            temp_700hpa = Sondetmp['drybulb_temp'][nearest_id_700hpa] # in K
            z_700hpa = Sondetmp['alt'][nearest_id_700hpa]*1.e3 # in meters
            tmp_sfc_temp = Sondetmp['drybulb_temp'][0] # in K
            tmp_sfc_pres = Sondetmp['pressure'][0] # in hPa
            tmp_sfc_rh = Sondetmp['RH'][0] # in %

            

            sfc_Moretmp = calculate_theta_and_more([tmp_sfc_temp],[tmp_sfc_pres],\
                                       RH=[tmp_sfc_rh],use_T_K=True,\
                                      sat_pres_formula='Emmanuel')

            tmp_sfc_theta = sfc_Moretmp['Theta'][0]
            lts = theta_700hpa - tmp_sfc_theta
            R_d = 287.058; # gas constant for dry air [J/(kg*K)].
            R_v = 461.5; # gas constant for water vapor [J/(kg*K)].
            c_p = 1005.7; # +- 2.5 [J/(kg*K)] - specific heat capacity of dry air at 273K in a constant pressure.
            g = 9.81 # gravitational acceleration, m/s^2
            
            dum_T = Sondetmp['drybulb_temp']
            dum_qs = Moretmp['q']*1.e-3
            dum_Lv = Moretmp['L_v']
            moist_ad_lapse_rate = (g/c_p) * (1 - ( (1 + ( (dum_Lv*dum_qs) / (R_d*dum_T) ) ) / (1 + ( ((dum_Lv**2.)*dum_qs) / (c_p*R_v*(dum_T**2.)) ) ) ) )
            #moist_ad_lapse_rate = (g/c_p) * (1 - ( (1+((Moretmp['L_v']*Moretmp['w_s'])/(R_d*Sondetmp['drybulb_temp']))) / (1 + (((Moretmp['L_v']**2.)*Moretmp['w_s'])/(c_p*R_v*(Sondetmp['drybulb_temp']**2.)))) ) ) # K/m
            #plt.plot(moist_ad_lapse_rate*1.e3,Sondetmp['alt'])
            nearest_val_850hpa,nearest_id_850hpa = find_nearest(Sondetmp['pressure'],850)
            #sfc_700hpa_temp_avg = (tmp_sfc_temp + temp_700hpa)/2.
            moist_ad_lapse_rate_850 = moist_ad_lapse_rate[nearest_id_850hpa] # K/m
            #moist_ad_lapse_rate_850 = moist_ad_lapse_rate_850*1.e3 # K/km
            pressure_metpy = Sondetmp['pressure'] * units.hPa
            temperature_metpy = Sondetmp['drybulb_temp'] * units.K
            rh_metpy = Sondetmp['RH'] * units.percent
            dewpoint_metpy = mpcalc.dewpoint_from_relative_humidity(temperature_metpy, rh_metpy)
            lcl_pres,lcl_temp = mpcalc.lcl(pressure_metpy[0],temperature_metpy[0],dewpoint_metpy[0])
            nearest_lcl_pres,nearest_lcl_pres_id = find_nearest(np.array(pressure_metpy),np.array(lcl_pres))
            lcl_z = Sondetmp['alt'][nearest_lcl_pres_id]*1.e3 # m
            eis = lts - moist_ad_lapse_rate_850*(z_700hpa-lcl_z)
            
            #if False:
            if True:
                print('eis:',eis,'K')
                print('lts:',lts,'K')
                print('lcl_z:',lcl_z,'m')
                print('z_700hpa:',z_700hpa,'m')
                print('moist_ad_lapse_rate_850:',moist_ad_lapse_rate_850,'K/m')
                print(' ')
            #print(aaa)
            
            
            native_sonde_dict[str(int(jj+1))] = {}
            native_sonde_dict[str(int(jj+1))]['temperature'] = Sondetmp['drybulb_temp']
            native_sonde_dict[str(int(jj+1))]['pressure'] = Sondetmp['pressure']
            native_sonde_dict[str(int(jj+1))]['rh'] = Sondetmp['RH']
            native_sonde_dict[str(int(jj+1))]['u'] = Sondetmp['u_wind']
            native_sonde_dict[str(int(jj+1))]['v'] = Sondetmp['v_wind']
            native_sonde_dict[str(int(jj+1))]['wind_speed'] = Sondetmp['wind_speed']
            native_sonde_dict[str(int(jj+1))]['wind_dir'] = Sondetmp['wind_direction']
            native_sonde_dict[str(int(jj+1))]['height'] = Sondetmp['alt']*1.e3
            native_sonde_dict[str(int(jj+1))]['time_dt_long'] = Sondetmp['time']


            native_sonde_dict[str(int(jj+1))]['q'] = Moretmp['q']
            native_sonde_dict[str(int(jj+1))]['theta'] = Moretmp['Theta']
            native_sonde_dict[str(int(jj+1))]['theta_e'] = Moretmp['Theta_e']
            native_sonde_dict[str(int(jj+1))]['rh_i'] = Moretmp['RH_i']
            native_sonde_dict[str(int(jj+1))]['w_s'] = Moretmp['L_v']
            native_sonde_dict[str(int(jj+1))]['e'] = Moretmp['e']
            native_sonde_dict[str(int(jj+1))]['time_dt'] = current_sonde_time
            native_sonde_dict[str(int(jj+1))]['cluster_id'] = current_cluster_id
            native_sonde_dict[str(int(jj+1))]['eis'] = eis
            native_sonde_dict[str(int(jj+1))]['lts'] = lts
            native_sonde_dict[str(int(jj+1))]['lcl'] = lcl_z
            
    if False:        
        for jj in range(len(sonde_flags)):
            if sonde_flags[jj]:
                fig = plt.figure(figsize=(18,9))
                dum_keys = ['temperature','rh','rh_i','theta','theta_e','pressure','wind_speed','wind_dir','u','v','q','w_s','e']
                for ii in range(len(dum_keys)):
                    dumax = fig.add_subplot(2,8,ii+1)
                    dumax.plot(native_sonde_dict[str(int(jj+1))][dum_keys[ii]],native_sonde_dict[str(int(jj+1))]['height']*1.e-3,c='k',lw=2)
                    dumax.set_ylabel('Height [km]',fontsize=14)
                    dumax.set_xlabel(dum_keys[ii],fontsize=14)
                    dumax.tick_params(labelsize=14)
                    dumax.grid(True)
                    plt.subplots_adjust(wspace=0.5,top=0.925)
                    plt.suptitle(native_sonde_dict[str(int(jj+1))]['time_dt'],fontsize=24)
                plt.show()
                plt.close()
    # We have a "native" sonde dict with the original soundings
    # Now calculate a few things
    if np.all(sonde_flags is not False):
        sonde_present_flag = True
    else:
        sonde_present_flag = False
    #return sonde_present_flag, native_sonde_dict
    return sonde_present_flag, native_sonde_dict

In [20]:
if False:
    eis_arr = []
    lts_arr = []
    time_arr = []
    target_date = datetime.datetime(2016,5,23)
    for in_date in basta_dates_dt:
        #print(in_date)
        native_sonde_present_flag, native_sonde_out_dict = process_native_sondes([in_date],sonde_dates_dt,sonde_times_dt,sonde_files,cluster_ids,cluster_times_dt)
        keys = list(native_sonde_out_dict.keys())
        for ii in range(len(keys)):
           # print(keys[ii])
            if native_sonde_out_dict[keys[ii]] is not None:
                eis_arr.append(native_sonde_out_dict[keys[ii]]['eis'])
                lts_arr.append(native_sonde_out_dict[keys[ii]]['lts'])
                time_arr.append(native_sonde_out_dict[keys[ii]]['time_dt'])
        #print(aaa)
    time_arr = np.array(time_arr)
    eis_arr = np.array(eis_arr)
    lts_arr = np.array(lts_arr)
    # remove duplicate values
    dum,dumid = np.unique(time_arr,return_index=True)
    time_arr = time_arr[dumid]
    eis_arr = eis_arr[dumid]
    lts_arr = lts_arr[dumid]
    
    sns.set_theme()
    #sns.set_style('dark')
    sns.set_style('ticks')
    sns.set(rc={'axes.facecolor':'lavender','axes.edgecolor': 'black','grid.color':'white'})
    sns.set_context('talk') 
    fig = plt.figure(figsize=(8,8))
    ax1 = fig.add_subplot(111)
    ax1.grid(True)
    Fontsize=16
    ax1.set_xlabel('LTS [K]',fontsize=Fontsize)
    ax1.set_ylabel('EIS [K]',fontsize=Fontsize)
    ax1.tick_params(labelsize=Fontsize)
    ax1.scatter(lts_arr,eis_arr,s=10,c='royalblue')
    ax1.set_xlim(0,25)
    ax1.set_ylim(-5,25)
    ax1.plot([0,25],[0,25],lw=3,c='black',ls='dashed')
    plt.show()
    print('done')

In [21]:
def process_optics(date,optics_dates_dt,optics_files,avg_bool,basta_time_ts):

    tmpid = np.where(optics_dates_dt == date)
    if np.size(tmpid) == 0.:
        optics_present_flag = False
        optics_dict = None
        return optics_present_flag,optics_dict
    elif np.size(tmpid) > 0.:
        optics_present_flag = True
        tmpid = tmpid[0][0]
        current_optics_file = optics_files[tmpid]
        current_optics_date = optics_dates_dt[tmpid]    

        ncfile = xarray.open_dataset(current_optics_file,decode_times=False)
        base_time = ncfile['base_time'].values
        time_offset = ncfile['time_offset'].values
        qc_time = ncfile['qc_time'].values
        tau_inst = ncfile['optical_depth_instantaneous'].values
        qc_tau_inst = ncfile['qc_optical_depth_instantaneous'].values
        reff_inst = ncfile['effective_radius_instantaneous'].values
        qc_reff_inst = ncfile['qc_effective_radius_instantaneous'].values
        tau_avg = ncfile['optical_depth_instantaneous'].values
        qc_tau_avg = ncfile['qc_optical_depth_average'].values
        reff_avg = ncfile['effective_radius_average'].values
        qc_reff_avg = ncfile['qc_effective_radius_average'].values
        trans1 = ncfile['total_transmittance_filter1'].values
        trans2 = ncfile['total_transmittance_filter2'].values
        trans3 = ncfile['total_transmittance_filter3'].values
        trans4 = ncfile['total_transmittance_filter4'].values
        trans5 = ncfile['total_transmittance_filter5'].values
        source_lwp = ncfile['source_lwp'].values
        lwp = ncfile['lwp'].values
        qc_lwp = ncfile['qc_lwp'].values
        pwv = ncfile['pwv'].values
        qc_pwv = ncfile['qc_pwv'].values
        cf = ncfile['cloudfraction'].values
        qc_cf = ncfile['qc_cloudfraction'].values
        ncfile.close()
        time_ts = base_time + time_offset
        time_dt = np.array([toDatetime(time_ts[dd]) for dd in range(len(time_ts))])
        
        tau_inst_interp = np.interp(basta_time_ts,time_ts,tau_inst)
        tau_avg_interp = np.interp(basta_time_ts,time_ts,tau_avg)
        trans1_interp = np.interp(basta_time_ts,time_ts,trans1)
        trans2_interp = np.interp(basta_time_ts,time_ts,trans2)
        trans3_interp = np.interp(basta_time_ts,time_ts,trans3)
        trans4_interp = np.interp(basta_time_ts,time_ts,trans4)
        trans5_interp = np.interp(basta_time_ts,time_ts,trans5)
        ref_inst_interp = np.interp(basta_time_ts,time_ts,reff_inst)
        reff_avg_interp = np.interp(basta_time_ts,time_ts,reff_avg)
        lwp_interp = np.interp(basta_time_ts,time_ts,lwp)
        cf_interp = np.interp(basta_time_ts,time_ts,cf)
        
        optics_dict = {'tau_inst':tau_inst_interp,\
                       'tau_avg':tau_avg_interp,\
                       'reff_inst':tau_avg_interp,\
                       'reff_avg':tau_avg_interp,\
                       'trans1':trans1_interp,\
                       'trans2':trans2_interp,\
                       'trans3':trans3_interp,\
                       'trans4':trans4_interp,\
                       'trans5':trans5_interp,\
                       'cloud_fraction':cf_interp,\
                       'lwp':lwp_interp,\
                       'native_tau_inst':tau_inst,\
                       'native_tau_avg':tau_avg,\
                       'native_trans1':trans1,\
                       'native_trans2':trans2,\
                       'native_trans3':trans3,\
                       'native_trans4':trans4,\
                       'native_trans5':trans5,\
                       'native_reff_inst':reff_inst,\
                       'native_reff_avg':reff_avg,\
                       'native_lwp':lwp,\
                       'native_lwp':pwv,\
                       'native_source_lwp':source_lwp,\
                       'native_time_dt':time_dt,\
                       'native_time_ts':time_ts,\
                       'native_cloud_fraction':cf,\
                      }
        
        return optics_present_flag,optics_dict

In [22]:
def interp_sondes(date,sonde_dates_dt,sonde_times_dt,sonde_files,basta_times_dt,basta_height,cluster_ids,cluster_times_dt):
    


    # Get current_date
    sonde_id = np.where(sonde_dates_dt == date)[0]

    # Pull out 3 potential soundings (1 on previous day, 2 on current_day)
    
    # The interpolation will work as follows:
    #   If there are 3 available soundings, times before 12 UTC will be
    #   linearly interpolated onto constant altitude levels between the
    #   12 UTC sounding and the previous 00 UTC sounding. Times after 12
    #   UTC are interpolated between 12 UTC and the 00 UTC sounding, which
    #  is generally at 2315 UTC. If there is a sounding at ~2315 UTC, the
    #  rest of the day is considerd to be consant with that sounding.
    
    date = date[0]
    
    target_time_1 = datetime.datetime(date.year,date.month,date.day,0)
    target_time_2 = datetime.datetime(date.year,date.month,date.day,12)
    tmp_time_delta = datetime.timedelta(days=1)
    target_time_3 = datetime.datetime(date.year,date.month,date.day,0) + tmp_time_delta
    
    sonde_times_dt = np.array(sonde_times_dt)
    
    tmpid = np.where((sonde_times_dt > (target_time_1 - datetime.timedelta(hours=2))) & (sonde_times_dt < (target_time_1 + datetime.timedelta(hours=2))))
    if np.size(tmpid) > 0:
        sonde_1_id = tmpid[0][0]
        sonde_1_flag = True
        sonde_1_time = sonde_times_dt[sonde_1_id]
    else:
        sonde_1_flag = False
        sonde_1_id = -999.
        sonde_1_time = -999.
    tmpid = np.where((sonde_times_dt > (target_time_2 - datetime.timedelta(hours=2))) & (sonde_times_dt < (target_time_2 + datetime.timedelta(hours=2))))
    if np.size(tmpid) > 0:
        sonde_2_id = tmpid[0][0]
        sonde_2_flag = True
        sonde_2_time = sonde_times_dt[sonde_2_id]
    else:
        sonde_2_flag = False
        sonde_2_id = -999.
        sonde_2_time = -999.
    tmpid = np.where((sonde_times_dt > (target_time_3 - datetime.timedelta(hours=2))) & (sonde_times_dt < (target_time_3 + datetime.timedelta(hours=2))))
    if np.size(tmpid) > 0:
        sonde_3_id = tmpid[0][0]
        sonde_3_flag = True
        sonde_3_time = sonde_times_dt[sonde_3_id]
    else:
        sonde_3_flag = False 
        sonde_3_id = -999.
        sonde_3_time = -999.
    
    sonde_flags = [sonde_1_flag,sonde_2_flag,sonde_3_flag]
    sonde_ids = [sonde_1_id,sonde_2_id,sonde_3_id]   

    sonde_dict = {}
    for jj in range(len(sonde_flags)):
        if not sonde_flags[jj]:
            sonde_dict[str(int(jj+1))] = None
            continue
        elif sonde_flags[jj]:
            current_sonde_id = sonde_ids[jj]
            current_sonde_file = sonde_files[current_sonde_id]
            current_sonde_time = sonde_times_dt[current_sonde_id]
            dumid = np.where(cluster_times_dt == current_sonde_time)
            if np.size(dumid) > 0.:
                current_cluster_id = cluster_ids[dumid[0][0]]
            else:
                current_cluster_id = None
            
            current_sonde_file = current_sonde_file.split('/')[-1]
            path = '/mnt/raid/mwstanfo/micre_data/micre_soundings/'
            file_size = os.stat(path+current_sonde_file).st_size/1.e3
            fstruct = fs(current_sonde_file,path,file_size)
            Sondetmp = load_sonde_data(fstruct)         

            max_alt = np.max(Sondetmp['alt'])
            if max_alt < 10.:
                print('Sonde failed to reach 10 km. Therefore omitting this sounding.')
                sonde_dict[str(int(jj+1))] = None
                sonde_flags[jj] = False
                continue
            else:
                pass    
    
            sonde_dict[str(int(jj+1))] = {}
            sonde_dict[str(int(jj+1))]['temperature'] = Sondetmp['drybulb_temp']    
            sonde_dict[str(int(jj+1))]['rh'] = Sondetmp['RH']    
            sonde_dict[str(int(jj+1))]['height'] = Sondetmp['alt']*1.e3
            sonde_dict[str(int(jj+1))]['time'] = current_sonde_time
            sonde_dict[str(int(jj+1))]['cluster_id'] = current_cluster_id
    
    
    num_times = len(basta_times_dt)
    num_heights = len(basta_height)
    basta_times_ts = np.array([toTimestamp(basta_times_dt[dd]) for dd in range(len(basta_times_dt))])

    dumid = np.where(sonde_flags)[0]
    num_current_soundings = np.size(dumid)
    
    print('# of current soundings:',num_current_soundings)
    if num_current_soundings == 0.:
        interp_sonde_present_flag = False
        interp_sonde_dict = None
        return interp_sonde_present_flag, interp_sonde_dict
    else:
        pass
    
    interp_sonde_present_flag = True
    interp_sonde_dict = None
    
    temp_arr = []
    rh_arr = []
    height_arr = []
    time_arr = []
    cluster_id_arr = []
    for jj in range(len(sonde_flags)):
        if sonde_flags[jj]:
            temp_arr.append(sonde_dict[str(int(jj+1))]['temperature'])
            rh_arr.append(sonde_dict[str(int(jj+1))]['rh'])
            height_arr.append(sonde_dict[str(int(jj+1))]['height'])
            time_arr.append(sonde_dict[str(int(jj+1))]['time'])
            cluster_id_arr.append(sonde_dict[str(int(jj+1))]['cluster_id'])
        else:
            continue

    temp_interp_arr = []
    rh_interp_arr = []
    for jj in range(num_current_soundings):
        temp_interp_arr.append(np.interp(basta_height,height_arr[jj],temp_arr[jj]))
        rh_interp_arr.append(np.interp(basta_height,height_arr[jj],rh_arr[jj]))
        
    temp_interp_arr = np.array(temp_interp_arr)
    rh_interp_arr = np.array(rh_interp_arr)
    time_arr = np.array(time_arr)
    time_arr_ts = np.array([toTimestamp(time_arr[dd]) for dd in range(len(time_arr))])
    cluster_id_arr = np.array(cluster_id_arr)
    
    # check interpolation
    if False:
        fig = plt.figure(figsize=(8,8))
        ax1 = fig.add_subplot(111)
        ax1.plot(temp_interp_arr[1],basta_height,c='b',lw=4)
        ax1.plot(temp_arr[1],height_arr[1],c='darkorange',ls='dotted',lw=4)
        ax1.set_ylim(0,np.max(basta_height))
        plt.show()
        plt.close()
        
    temp_interp = []
    rh_interp = []
    for kk in range(num_heights):
        temp_interp.append(np.interp(basta_times_ts,time_arr_ts,temp_interp_arr[:,kk],\
                                                       left=temp_interp_arr[0,kk],\
                                                       right=temp_interp_arr[-1,kk]))        
        rh_interp.append(np.interp(basta_times_ts,time_arr_ts,rh_interp_arr[:,kk],\
                                                       left=rh_interp_arr[0,kk],\
                                                       right=rh_interp_arr[-1,kk])) 


    temp_interp = np.array(temp_interp)
    rh_interp = np.array(rh_interp)
    
    if False:
        fig = plt.figure(figsize=(8,6))
        ax1= fig.add_subplot(211)
        ax2= fig.add_subplot(212)
        
        dumplot = ax1.contourf(basta_times_dt,basta_height,temp_interp-273.15,cmap='RdYlBu_r')
        cbar = fig.colorbar(dumplot,ax=ax1)
        cbar.ax.set_ylabel('Temperature [$^{\\circ}$]')
        dumplot = ax2.contourf(basta_times_dt,basta_height,rh_interp,cmap='ocean')
        cbar = fig.colorbar(dumplot,ax=ax2)
        cbar.ax.set_ylabel('RH [%]')
                        
        ax1.grid(True)
        ax2.grid(True)
        plt.show()
        plt.close()
    

    # calculate time to nearest sounding
    time_to_nearest_sounding = []
    nearest_cluster_id = []
    # Now calculate time to nearest sounding using timedeltas
    for ttt in range(len(basta_times_dt)):
        single_time_ts = toTimestamp(basta_times_dt[ttt])

        diff_arr = np.abs(single_time_ts - time_arr_ts)
        dum_min = np.min(diff_arr)
        time_to_nearest_sounding.append(dum_min)
        
        # Find nearest sounding
        nearest_val,nearest_id = find_nearest(time_arr_ts,single_time_ts)
        nearest_cluster_id.append(cluster_id_arr[nearest_id])
    time_to_nearest_sounding = np.array(time_to_nearest_sounding)
    nearest_cluster_id = np.array(nearest_cluster_id)
    
    # Boolean plot to check cluster id "interpolation"
    if False:
        fig = plt.figure(figsize=(12,4))
        ax = fig.add_subplot(111)
        ax.plot(basta_times_dt,nearest_cluster_id)
        plt.show()
        plt.close()
                                  
                                
                        
    interp_sonde_dict = {'temperature':temp_interp,'rh':rh_interp,'seconds_to_nearest_sounding':time_to_nearest_sounding,'nearest_cluster_id':nearest_cluster_id}
    
    return interp_sonde_present_flag,interp_sonde_dict

In [23]:
# for date in basta_dates_dt[13:]:
#     in_date = [date]
#     basta_present_flag,merged_dict = merge_instruments(in_date,basta_files,basta_dates_dt,False,arm_ceil_dates_dt,arm_ceil_files,aad_ceil_dates_dt,aad_ceil_files,sonde_dates_dt,sonde_files)
#     print(aaaa)
#     if basta_present_flag:
#         save_path = '/mnt/raid/mwstanfo/micre/merged_instrument_files/'
#         dum_time_str = in_date[0].strftime('%Y%m%d')
#         out_pkl_file = save_path+'merged_instruments_{}_v2.p'.format(dum_time_str)
#         pickle.dump(merged_dict,open(out_pkl_file,"wb"))
#         print(aaaa)

In [24]:
#==================================================
# Function to process basta data
#
# Inputs: date -- from basta_dates_dt
#         avg_bool -- boolean to perform averaging
#==================================================
def process_basta(date,basta_dates_dt,basta_files,avg_bool):
    # grab date
    dumid = np.where(basta_dates_dt == date[0])
    dumid = dumid[0][0]
    
    # read in file
    ncfile = xarray.open_dataset(basta_files[dumid],decode_times=False)
    basta_time_dims = ncfile.dims['time'] # will be variable according to up-time
    basta_height_dims = ncfile.dims['height'] # should always be 480
    basta_ref = np.array(ncfile['reflectivity'].copy())
    basta_vel = np.array(ncfile['velocity'].copy())
    basta_flag = np.array(ncfile['flag'].copy())
    basta_flag_coupling = np.array(ncfile['flag_coupling'].copy()) # 0: no coupling (good); 1: coupling (bad)
    basta_noise_level = np.array(ncfile['noise_level'].copy()) # 0: good data; 1-9: bad data; -1: no data
    basta_time_sec_since_00Z = np.array(ncfile['time'].copy())
    basta_height = np.array(ncfile['height'].copy()) # 25-m resolution beginning at 12.5 m (mid-bin)
    ### ends at 11987.5 m, so 12 km
    ncfile.close()
    
    tmp_basta_time_ts = toTimestamp(datetime.datetime(date[0].year,\
                                       date[0].month,\
                                       date[0].day))
    tmp_basta_time_ts = tmp_basta_time_ts + basta_time_sec_since_00Z
    basta_time_dt = [toDatetime(tmp_basta_time_ts[dd]) for dd in range(len(tmp_basta_time_ts))]
    basta_time_dt = np.array(basta_time_dt) # holds the BASTA time array for the current file.
    
    # Need to ensure BASTA times are stricty increasing.
    # This is a rare occurrence--only 2 days. Informed
    # Alain Protat, but just omitting these cases.
    diff = np.diff(tmp_basta_time_ts)

    if np.min(diff) < 0.:
        #raise RuntimeError('BASTA times are not strictly increasing.')
        basta_present_flag = False
        basta_out_dict = None
        return basta_present_flag, basta_out_dict
    else:
        pass
    
    basta_present_flag = True
    
    #------------------------------------------------------
    # For some of the files, the date after the current one
    # holds the last hour of the day. In these cases, will
    # need to pull in the following day.
    #------------------------------------------------------
    
    # Check first time of following file and grab times and associated
    # variables where the DAY matches that of the current BASTA day.
    # Grab next file
    if (date[0] != datetime.datetime(2016,4,2)) and (date[0] != datetime.datetime(2017,3,17)):

        ncfile = xarray.open_dataset(basta_files[dumid+1],decode_times=False)

        after_basta_time_sec_since_00Z = np.array(ncfile['time'].copy())
        ncfile.close()
        tmp_basta_time_ts = toTimestamp(datetime.datetime(basta_dates_dt[dumid+1].year,\
                                           basta_dates_dt[dumid+1].month,\
                                           basta_dates_dt[dumid+1].day))

        tmp_basta_time_ts = tmp_basta_time_ts + after_basta_time_sec_since_00Z
        after_basta_time_dt = [toDatetime(tmp_basta_time_ts[dd]) for dd in range(len(tmp_basta_time_ts))]
        after_basta_date_dt = [datetime.datetime(after_basta_time_dt[dd].year,\
                                                after_basta_time_dt[dd].month,\
                                                after_basta_time_dt[dd].day) for dd in range(len(after_basta_time_dt))]
        after_basta_time_dt = np.array(after_basta_time_dt) # holds the BASTA time array for the after file.
        after_basta_date_dt = np.array(after_basta_date_dt) # holds the BASTA date array for the after file.
        
        # check to see if any of the dates in the after file equal the date on the current file
        date = date[0]
        tmpid = np.where(after_basta_date_dt == date)
        if np.size(tmpid) > 0.:
            # now open back up after file and add indices in after file with
            # same date as current file to the current BASTA arrays
            ncfile = xarray.open_dataset(basta_files[dumid+1],decode_times=False)
            after_basta_ref = np.array(ncfile['reflectivity'].copy())
            after_basta_vel = np.array(ncfile['velocity'].copy())
            after_basta_flag = np.array(ncfile['flag'].copy())
            after_basta_flag_coupling = np.array(ncfile['flag_coupling'].copy()) # 0: no coupling (good); 1: coupling (bad)
            after_basta_noise_level = np.array(ncfile['noise_level'].copy()) # 0: good data; 1-9: bad data; -1: no data
            ncfile.close()  
            
            # now concatenate arrays
            basta_time_dt = np.concatenate((basta_time_dt,after_basta_time_dt[tmpid]))
            basta_ref = np.concatenate((basta_ref,np.squeeze(after_basta_ref[:,tmpid])),axis=1)
            basta_vel = np.concatenate((basta_vel,np.squeeze(after_basta_vel[:,tmpid])),axis=1)
            basta_flag = np.concatenate((basta_flag,np.squeeze(after_basta_flag[:,tmpid])),axis=1)
            basta_flag_coupling = np.concatenate((basta_flag_coupling,after_basta_flag_coupling[tmpid]),axis=0)
            basta_noise_level = np.concatenate((basta_noise_level,after_basta_noise_level[tmpid]),axis=0)
            

                
    # Because the current date BASTA file sometimes start at 23Z on the day prior, also need to
    # limit the current file to encompass only times on the current date (i.e., need to limit
    # the current date variables, filtering out those from 23Z-00Z on the previous date)
    basta_date_dt = np.array([datetime.datetime(basta_time_dt[dd].year,\
                                                basta_time_dt[dd].month,\
                                                basta_time_dt[dd].day) for dd in range(len(basta_time_dt))])
    
    # Check to see if any of the dates in the current file equal the before date
    if (date != datetime.datetime(2016,4,2)) and (date != datetime.datetime(2017,3,17)):
        tmpid = np.where(basta_date_dt != date)
        if np.size(tmpid) > 0.:
            # limit arrays
            tmpid = np.where(basta_date_dt == date)
            basta_time_dt = basta_time_dt[tmpid]
            basta_flag_coupling = basta_flag_coupling[tmpid]
            basta_flag = basta_flag[:,tmpid]
            basta_ref = basta_ref[:,tmpid]
            basta_vel = basta_vel[:,tmpid]
            basta_noise_level = basta_noise_level[tmpid]

    
    basta_ref = np.squeeze(basta_ref)
    basta_vel = np.squeeze(basta_vel)
    basta_flag = np.squeeze(basta_flag)
    basta_flag_coupling = np.squeeze(basta_flag_coupling)
    basta_noise_level = np.squeeze(basta_noise_level)         
        
        
    #-----------------------------------------
    #-----------------------------------------
    #-----------------------------------------
    # Create a series of flags that will indicate
    # whether or not a cloud is present
    # 
    # basta_flag == 1 means radar is working
    # properly but there is no data. These values
    # are flagged as -999. When basta_flag > 0.
    # or when basta_flag_coupling > 0., we will
    # assign these values as NaNs. For values
    # below the theoretical minimum detectable signal,
    # we will assign these as -999. as well. Values
    # up to 137.5 m will be flagged as NaNs, indicating
    # bad data.
    #-----------------------------------------
    #-----------------------------------------
    #-----------------------------------------
    
    bad_radar_data_flag = np.zeros(len(basta_time_dt))
    min_basta_loc = 6
    # create array that is the minimum detectable signal as a function of altitude
    Z_min_1km = -36.
    ref_range = 1000.
    Z_min = Z_min_1km + 20.*np.log10(basta_height) - 20.*np.log10(ref_range)
    Z_min[0] = -999.    

    # NaN out values up to 137.5 m due to surface clutter
    basta_ref[0:min_basta_loc,:] = np.nan
    basta_vel[0:min_basta_loc,:] = np.nan
    # We will also assign all basta_flag values up to 137.5 m
    # as -1. Currently basta_flag == -1 only up to 87.5 m, so we
    # want to adjust this.
    basta_flag[0:min_basta_loc,:] = -1
            
        
    # Set values below the minimum detectable signal to -999.
    for ttt in range(len(basta_time_dt)):
        dumid = np.where(basta_ref[:,ttt] < Z_min)
        if np.size(dumid) > 0.:
            basta_ref[dumid,ttt] = -999.
            basta_vel[dumid,ttt] = -999.
            basta_flag[dumid,ttt] = -1
    
    dumid = np.where(basta_flag_coupling == 1.)
    if np.size(dumid) > 0.:
        basta_ref[:,dumid] = np.nan
        basta_vel[:,dumid] = np.nan
        bad_radar_data_flag[dumid] = 1
        
    for ttt in range(len(basta_time_dt)):
        single_time_basta_flag = basta_flag[:,ttt]
        dumid = np.where(single_time_basta_flag > 0.)
        if np.size(dumid) > 0.:
            basta_ref[dumid,ttt] = np.nan
            basta_vel[dumid,ttt] = np.nan
        if np.all(single_time_basta_flag > 0.):
            bad_radar_data_flag[ttt] = 1           
    
    basta_time_ts = np.array([toTimestamp(basta_time_dt[dd]) for dd in range(len(basta_time_dt))])
    basta_out_dict = {'time_dt':basta_time_dt,\
                      'time_ts':basta_time_ts,\
                      'ref':basta_ref,\
                      'vel':basta_vel,\
                      'height':basta_height,\
                      'bad_radar_data_flag':bad_radar_data_flag,\
                      'flag':basta_flag,\
                     }
    
    return basta_present_flag,basta_out_dict

In [25]:
def process_arm_ceil(date,arm_ceil_dates_dt,arm_ceil_files,avg_bool,basta_height,basta_time_ts,interp_backscatter=False):
    tmpid = np.where(arm_ceil_dates_dt == date)
    if np.size(tmpid) == 0.:
        arm_ceil_present_flag = False
        arm_ceil_out_dict = None
        return arm_ceil_present_flag, arm_ceil_out_dict
    elif np.size(tmpid) > 0.:        
        arm_ceil_present_flag = True
        tmpid = tmpid[0][0]
        current_ceil_file = arm_ceil_files[tmpid]
        ncfile = xarray.open_dataset(current_ceil_file,decode_times=False)
        ceil_dims = ncfile.dims
        ceil_base_time = np.array(ncfile['base_time'].copy())        
        ceil_num_times = ceil_dims['time']
        ceil_time_offset = np.array(ncfile['time_offset'].copy())
        ceil_cbh_1 = np.array(ncfile['first_cbh'].copy())
        ceil_qc_cbh_1 = np.array(ncfile['qc_first_cbh'].copy())
        ceil_cbh_2 = np.array(ncfile['second_cbh'].copy())
        ceil_qc_cbh_2 = np.array(ncfile['qc_second_cbh'].copy())
        ceil_cbh_3 = np.array(ncfile['third_cbh'].copy())
        ceil_qc_cbh_3 = np.array(ncfile['qc_third_cbh'].copy())
        ceil_backscatter = np.array(ncfile['backscatter'].copy())
        ceil_sum_backscatter = np.array(ncfile['sum_backscatter'].copy())
        ceil_qc_sum_backscatter = np.array(ncfile['qc_sum_backscatter'].copy())
        ceil_detection_status = np.array(ncfile['detection_status'].copy())
        ceil_time_ts = [int(ceil_base_time + ceil_time_offset[dd]) for dd in range(ceil_num_times)]
        ceil_time_dt = [toDatetime(ceil_time_ts[dd]) for dd in range(ceil_num_times)]    
        ceil_range_bounds = np.array(ncfile['range_bounds'].copy())
        ceil_range = np.array(ncfile['range'].copy())
        ncfile.close()

        # qc backscatter
        dumxid = np.where(ceil_qc_sum_backscatter > 0)
        if np.size(dumxid) > 0.:
            #dumxid = np.squeeze(dumxid)
            ceil_backscatter[dumxid,:] = np.nan 
            ceil_sum_backscatter[dumxid,:] = np.nan 
        ceil_backscatter = ceil_backscatter*1.e-3/10000.  
        
        ceil_backscatter_log = ceil_backscatter.copy()
        dumid = np.where((~np.isnan(ceil_backscatter)) & (ceil_backscatter > 0.) )
        ceil_backscatter_log[dumid] = np.log10(ceil_backscatter[dumid])
        dumid = np.where(ceil_backscatter == 0.)
        ceil_backscatter_log[dumid] = np.nan
        dumid = np.where(ceil_backscatter < 0.)
        ceil_backscatter_log[dumid] = np.nan
        #ceil_backscatter_log[np.isnan(ceil_backscatter_log)] = 0. 
        ceil_backscatter = ceil_backscatter_log

        #------------------------------------------
        # Interpolate ceilometer to radar time grid
        # using nearest neighbor interpolation. 
        # This method requires that the nearest
        # neighbor be within 15 seconds of the
        # radar time grid element.
        #------------------------------------------
        ceil_time_ts = np.array([toTimestamp(ceil_time_dt[dd]) for dd in range(len(ceil_time_dt))])
        basta_bin_edges = np.arange(0,np.max(basta_height)+12.5+25.,25.)
        basta_height = np.around(basta_height,2)
        basta_time_dt = np.array([toDatetime(basta_time_ts[dd]) for dd in range(len(basta_time_ts))])
        ceil_cbh_1_interp = []
        ceil_cbh_2_interp = []
        ceil_cbh_3_interp = []
        ceil_detection_status_interp = []
        ceil_cbh_bin_relative_interp = []
        for ttt in range(len(basta_time_ts)):
            # if here, then good radar data exists
            # Now find the nearest in time ceilometer time step to the radar time step
            # If the ceilometer is more than 15 seconds away from the the radar time step,
            # then we will flag it as missing data (NaN)
            nearest_val,nearest_id = find_nearest(ceil_time_ts,basta_time_ts[ttt])
            time_diff = np.abs(nearest_val - basta_time_ts[ttt])
            target_time_diff = 16
            if time_diff <= target_time_diff:
                nearest_ceil_cbh_1 = ceil_cbh_1[nearest_id]
                nearest_ceil_cbh_2 = ceil_cbh_2[nearest_id]
                nearest_ceil_cbh_3 = ceil_cbh_3[nearest_id]
                nearest_ceil_detection_status = ceil_detection_status[nearest_id]
                ceil_detection_status_interp.append(nearest_ceil_detection_status)

                if np.isnan(nearest_ceil_detection_status):
                    #ceil_detection_status_interp.append(np.nan)
                    ceil_cbh_1_interp.append(np.nan)
                    ceil_cbh_2_interp.append(np.nan)
                    ceil_cbh_3_interp.append(np.nan)
                    ceil_cbh_bin_relative_interp.append(np.nan)
                    continue
                    
                elif (nearest_ceil_detection_status == 5.) or (nearest_ceil_detection_status == 0.):
                    ceil_cbh_1_interp.append(-999.)
                    ceil_cbh_2_interp.append(-999.)
                    ceil_cbh_3_interp.append(-999.)
                    ceil_cbh_bin_relative_interp.append(-999.)
                    continue
                    
                elif (nearest_ceil_detection_status == 4.):
                    ceil_cbh_1_interp.append(np.nan)
                    ceil_cbh_2_interp.append(np.nan)
                    ceil_cbh_3_interp.append(np.nan)
                    ceil_cbh_bin_relative_interp.append(np.nan)
                    continue                    
                    
                elif (nearest_ceil_detection_status == 1.) or (nearest_ceil_detection_status == 2.) or (nearest_ceil_detection_status == 3.):
                    pass
                else:
                    raise RuntimeError('Something went wrong')

                #if np.isnan(nearest_ceil_cbh_1):
                #    ceil_cbh_1_interp.append(np.nan)
                #    ceil_cbh_2_interp.append(np.nan)
                #    ceil_cbh_3_interp.append(np.nan)
                #    ceil_cbh_bin_relative_interp.append(np.nan)
                #    continue
                
                if np.isnan(nearest_ceil_cbh_1):
                    print(nearest_ceil_cbh_1)
                    print(nearest_ceil_detection_status)
                    raise RuntimeError('Something went wrong')
                    
                nearest_val,nearest_id = find_nearest(basta_bin_edges,nearest_ceil_cbh_1)
                if nearest_ceil_cbh_1 == nearest_val:
                    bin_edges = basta_bin_edges[nearest_id-1:nearest_id+1]
                    midbin = (bin_edges[0]+bin_edges[1])/2.
                    ceil_cbh_1_interp.append(midbin)
                    ceil_cbh_bin_relative_interp.append(1.)
                elif nearest_ceil_cbh_1 < nearest_val:
                    bin_edges = basta_bin_edges[nearest_id-1:nearest_id+1]
                    midbin = (bin_edges[0]+bin_edges[1])/2.
                    ceil_cbh_1_interp.append(midbin)
                    ceil_cbh_bin_relative_interp.append(0.)
                elif nearest_ceil_cbh_1 > nearest_val:
                    bin_edges = basta_bin_edges[nearest_id:nearest_id+2]
                    midbin = (bin_edges[0]+bin_edges[1])/2.
                    ceil_cbh_1_interp.append(midbin)
                    ceil_cbh_bin_relative_interp.append(2.)
                    
                if np.isnan(nearest_ceil_cbh_2):
                    ceil_cbh_2_interp.append(np.nan)
                else:    
                    nearest_val,nearest_id = find_nearest(basta_bin_edges,nearest_ceil_cbh_2)
                    if nearest_ceil_cbh_2 == nearest_val:
                        bin_edges = basta_bin_edges[nearest_id-1:nearest_id+1]
                        midbin = (bin_edges[0]+bin_edges[1])/2.
                        ceil_cbh_2_interp.append(midbin)
                    elif nearest_ceil_cbh_2 < nearest_val:
                        bin_edges = basta_bin_edges[nearest_id-1:nearest_id+1]
                        midbin = (bin_edges[0]+bin_edges[1])/2.
                        ceil_cbh_2_interp.append(midbin)
                    elif nearest_ceil_cbh_2 > nearest_val:
                        bin_edges = basta_bin_edges[nearest_id:nearest_id+2]
                        midbin = (bin_edges[0]+bin_edges[1])/2.
                        ceil_cbh_2_interp.append(midbin)

                if np.isnan(nearest_ceil_cbh_3):
                    ceil_cbh_3_interp.append(np.nan)
                else:                      
                    nearest_val,nearest_id = find_nearest(basta_bin_edges,nearest_ceil_cbh_3)
                    if nearest_ceil_cbh_3 == nearest_val:
                        bin_edges = basta_bin_edges[nearest_id-1:nearest_id+1]
                        midbin = (bin_edges[0]+bin_edges[1])/2.
                        ceil_cbh_3_interp.append(midbin)
                    elif nearest_ceil_cbh_3 < nearest_val:
                        bin_edges = basta_bin_edges[nearest_id-1:nearest_id+1]
                        midbin = (bin_edges[0]+bin_edges[1])/2.
                        ceil_cbh_3_interp.append(midbin)
                    elif nearest_ceil_cbh_3 > nearest_val:
                        bin_edges = basta_bin_edges[nearest_id:nearest_id+2]
                        midbin = (bin_edges[0]+bin_edges[1])/2.
                        ceil_cbh_3_interp.append(midbin)                    
                    
                    
                    
            else:
                #print('here')
                #print(time_diff,basta_time_dt[ttt],ceil_time_dt[nearest_id])
                ceil_cbh_1_interp.append(np.nan)
                ceil_cbh_2_interp.append(np.nan)
                ceil_cbh_3_interp.append(np.nan)
                ceil_detection_status_interp.append(np.nan)
                ceil_cbh_bin_relative_interp.append(np.nan)

        ceil_cbh_1_interp = np.array(ceil_cbh_1_interp)
        ceil_cbh_2_interp = np.array(ceil_cbh_2_interp)
        ceil_cbh_3_interp = np.array(ceil_cbh_3_interp)
        ceil_detection_status_interp = np.array(ceil_detection_status_interp)    
        ceil_cbh_bin_relative_interp = np.array(ceil_cbh_bin_relative_interp)  
        
        
        #============================================
        # Plot to explore backscatter interpolation
        #============================================

        #if True:
        if False:

            fig = plt.figure(figsize=(24,14))
            Fontsize=14
            dfmt = mdates.DateFormatter('%H:%M')
            ax_native = fig.add_subplot(211)
            ax_interp_nn = fig.add_subplot(212)
            #ax_interp_cubic1 = fig.add_subplot(223)
            #ax_interp_linear = fig.add_subplot(224)
            
            basta_time_dt = np.array([toDatetime(basta_time_ts[dd]) for dd in range(len(basta_time_ts))])             
            start_time = datetime.datetime(basta_time_dt[0].year,basta_time_dt[0].month,basta_time_dt[0].day,0,0)
            #end_time = start_time + datetime.timedelta(days=1)           
            end_time = start_time + datetime.timedelta(hours=24)           
            
            
            #axlist = [ax_native,ax_interp_nn,ax_interp_cubic1,ax_interp_linear]
            axlist = [ax_native,ax_interp_nn]
            for ax in axlist:
                ax.tick_params(labelsize=Fontsize)
                ax.set_ylabel('Height [km]',fontsize=Fontsize)
                ax.set_xlabel('UTC Time [HH:MM]',fontsize=Fontsize)
                ax.xaxis.set_major_formatter(dfmt)
                ax.grid(which='both',c='dimgrey',ls='dotted',lw=1)
                ax.set_xlim(start_time,end_time)
                ax.set_ylim(0,1)
                
            cmap = matplotlib.cm.get_cmap("jet").copy()
            cmap.set_under('navy')
            cmap.set_bad('grey')

            # Native
            height_bins = ceil_range_bounds[:,0]
            dumbin = np.array([height_bins[-1]+30])
            height_bins = np.concatenate((height_bins,dumbin))

            native_plot = ax_native.pcolormesh(ceil_time_dt,\
                                                             height_bins*1.e-3,\
                                                             ceil_backscatter[1:,:].T,\
                                                             cmap=cmap,
                                                             vmin=-8,vmax=-3)
            # Colorbar
            dum_ticks = [-8,-7,-6,-5,-4,-3]
            native_cbar = fig.colorbar(native_plot,ticks=dum_ticks,pad=0.01,ax=ax_native)
            dumstr = '$log_{10}$($\\beta_{att}$)'
            native_cbar.ax.set_ylabel(dumstr,fontsize=Fontsize)
            native_cbar.ax.tick_params(labelsize=Fontsize)  
            
            ax_native.set_title('Native $\\beta_{att}$ \n 30-m x 16-sec resolution',fontsize=Fontsize*1.5,color='dimgrey')
            ax_native.scatter(ceil_time_dt,ceil_cbh_1*1.e-3,s=2,c='black')

            
            # Fill in obscured time periods with transparent red
            id4 = np.where(ceil_detection_status == 4.)
            detection_mask = np.zeros(np.shape(ceil_detection_status))
            if np.size(id4) > 1.:
                id4 = np.squeeze(id4)
                detection_mask[id4] = 1
                detection_Objects,num_detection_objects = ndimage.label(detection_mask)
                for kk in range(num_detection_objects):
                    dumid = np.where(detection_Objects == kk+1)[0]
                    first_id = dumid[0]
                    last_id = dumid[-1]
                    ax_native.axvspan(ceil_time_dt[first_id],\
                                ceil_time_dt[last_id],color='red',alpha=0.5)               
            
            
            
            ceil_time_dt = np.array(ceil_time_dt)
            ceil_time_dt_orig = ceil_time_dt.copy()
            ceil_time_ts_orig = ceil_time_ts.copy()
            
            dumid = np.where( (ceil_time_ts >= basta_time_ts[0]) & (ceil_time_ts <= basta_time_ts[-1]))
            if np.size(dumid) > 0.:
                dumid = np.squeeze(dumid)
                ceil_time_ts = ceil_time_ts[dumid]
                ceil_time_dt = ceil_time_dt[dumid]
                ceil_backscatter = ceil_backscatter[dumid,:]

            # Interpolated in log10 space (nearest neighbor)            
            x=ceil_time_ts = np.array([toTimestamp(ceil_time_dt[dd]) for dd in range(len(ceil_time_dt))])
            y=ceil_range
            #mask invalid values
            z = ceil_backscatter.copy()
            z = z.T
            xx, yy = np.meshgrid(x, y)
            basta_height_lim = basta_height[basta_height < np.max(ceil_range)]
            newy = basta_height_lim
            newx = basta_time_ts
            
            newX,newY = np.meshgrid(newx,newy)
            ceil_backscatter_interp_nn = griddata((xx.ravel(), yy.ravel()), z.ravel(),(newX, newY),method='nearest',fill_value=np.nan)
        
            dumid = np.where( (basta_time_ts < ceil_time_ts_orig[0]) | (basta_time_ts > ceil_time_ts_orig[-1]) )
            if np.size(dumid) > 0.:
                dumid = np.squeeze(dumid)
                ceil_cbh_1_interp[dumid] = np.nan
                ceil_cbh_2_interp[dumid] = np.nan
                ceil_cbh_3_interp[dumid] = np.nan
                ceil_backscatter_interp_nn[:,dumid] = np.nan
                ceil_detection_status_interp[dumid] = np.nan
                ceil_cbh_bin_relative_interp[dumid] = np.nan   

                 
            # Need to deal with missing times in between the start and end time of the radar
            # Let's loop through basta times and if the time is not within 12 seconds of a
            # radar profile, we'll nan it out

            for dum_tt in range(len(basta_time_dt)):
                dum_diff = np.abs(basta_time_ts[dum_tt] - ceil_time_ts)
                if np.min(dum_diff) > 12.:
                    ceil_backscatter_interp_nn[:,dum_tt] = np.nan               
                
                
                
            basta_height_bins = np.arange(0,np.max(basta_height_lim),25)
            dumbins = np.array([np.max(basta_height_lim)+12.5])
            basta_height_bins = np.concatenate((basta_height_bins,dumbins))
            interp_nn_plot = ax_interp_nn.pcolormesh(basta_time_dt,\
                                                             basta_height_bins*1.e-3,\
                                                             ceil_backscatter_interp_nn[:,1:],\
                                                             cmap=cmap,
                                                             vmin=-8,vmax=-3)      
            

            # Colorbar
            dum_ticks = [-8,-7,-6,-5,-4,-3]
            interp_nn_cbar = fig.colorbar(interp_nn_plot,ticks=dum_ticks,pad=0.01,ax=ax_interp_nn)
            dumstr = '$log_{10}$($\\beta_{att}$)'
            interp_nn_cbar.ax.set_ylabel(dumstr,fontsize=Fontsize)
            interp_nn_cbar.ax.tick_params(labelsize=Fontsize)  
            
            ax_interp_nn.set_title('Nearest Neighbor Interpolation $\\beta_{att}$ \n 25-m x 12-sec resolution',fontsize=Fontsize*1.5,color='dimgrey')            
            # Interpolated CBH
            ax_interp_nn.scatter(basta_time_dt,ceil_cbh_1_interp*1.e-3,s=2,c='black')

            
            # Fill in obscured time periods with transparent red
            id4 = np.where(ceil_detection_status_interp == 4.)
            detection_mask = np.zeros(np.shape(ceil_detection_status_interp))
            if np.size(id4) > 1.:
                id4 = np.squeeze(id4)
                detection_mask[id4] = 1
                detection_Objects,num_detection_objects = ndimage.label(detection_mask)
                for kk in range(num_detection_objects):
                    dumid = np.where(detection_Objects == kk+1)[0]
                    first_id = dumid[0]
                    last_id = dumid[-1]
                    ax_interp_nn.axvspan(basta_time_dt[first_id],\
                                basta_time_dt[last_id],color='red',alpha=0.5)               
            
            red_patch = mpatches.Patch(color='red',alpha=0.5,label='CEIL obscured')
            lgnd = ax_interp_nn.legend(handles=[red_patch],\
                                fontsize=Fontsize*1.5,\
                                bbox_to_anchor=(1,1.2),\
                                ncol=1,loc='upper right',framealpha=0)

            if False:
                # Cubic Interpolation
                x=ceil_time_ts = np.array([toTimestamp(ceil_time_dt[dd]) for dd in range(len(ceil_time_dt))])
                y=ceil_range
                #mask invalid values
                z = ceil_backscatter.copy()
                dumid = np.where(~np.isnan(z))
                z[dumid] = 10.**z[dumid]
                dumid = np.where(np.isnan(z))
                z[dumid] = 0.
                z = z.T
                xx, yy = np.meshgrid(x, y)
                basta_height_lim = basta_height[basta_height < np.max(ceil_range)]
                newy = basta_height_lim
                newx = basta_time_ts
                newX,newY = np.meshgrid(newx,newy)
                ceil_backscatter_interp_cubic1 = griddata((xx.ravel(), yy.ravel()), z.ravel(),(newX, newY),method='cubic',fill_value=np.nan)
                dumid = np.where(~np.isnan(ceil_backscatter_interp_cubic1))
                ceil_backscatter_interp_cubic1[dumid] = np.log10(ceil_backscatter_interp_cubic1[dumid])


                basta_height_bins = np.arange(0,np.max(basta_height_lim),25)
                dumbins = np.array([np.max(basta_height_lim)+12.5])
                basta_height_bins = np.concatenate((basta_height_bins,dumbins))
                interp_cubic1_plot = ax_interp_cubic1.pcolormesh(basta_time_dt,\
                                                                 basta_height_bins*1.e-3,\
                                                                 ceil_backscatter_interp_cubic1[:,1:],\
                                                                 cmap=cmap,
                                                                 vmin=-8,vmax=-3)            

                # Colorbar
                dum_ticks = [-8,-7,-6,-5,-4,-3]
                interp_cubic1_cbar = fig.colorbar(interp_cubic1_plot,ticks=dum_ticks,pad=0.01,ax=ax_interp_cubic1)
                dumstr = '$log_{10}$($\\beta_{att}$)'
                interp_cubic1_cbar.ax.set_ylabel(dumstr,fontsize=Fontsize)
                interp_cubic1_cbar.ax.tick_params(labelsize=Fontsize)  

                ax_interp_cubic1.set_title('Cubic Interpolation $\\beta_{att}$ \n 25-m x 12-sec resolution',fontsize=Fontsize*1.5,color='dimgrey')            


                # Linear Interpolation
                x=ceil_time_ts = np.array([toTimestamp(ceil_time_dt[dd]) for dd in range(len(ceil_time_dt))])
                y=ceil_range
                #mask invalid values
                z = ceil_backscatter.copy()
                dumid = np.where(~np.isnan(z))
                z[dumid] = 10.**z[dumid]
                dumid = np.where(np.isnan(z))
                z[dumid] = 0.
                z = z.T
                xx, yy = np.meshgrid(x, y)
                basta_height_lim = basta_height[basta_height < np.max(ceil_range)]
                newy = basta_height_lim
                newx = basta_time_ts
                newX,newY = np.meshgrid(newx,newy)
                ceil_backscatter_interp_linear = griddata((xx.ravel(), yy.ravel()), z.ravel(),(newX, newY),method='linear',fill_value=np.nan)
                dumid = np.where(~np.isnan(ceil_backscatter_interp_linear))
                ceil_backscatter_interp_linear[dumid] = np.log10(ceil_backscatter_interp_linear[dumid])


                basta_height_bins = np.arange(0,np.max(basta_height_lim),25)
                dumbins = np.array([np.max(basta_height_lim)+12.5])
                basta_height_bins = np.concatenate((basta_height_bins,dumbins))
                interp_linear_plot = ax_interp_linear.pcolormesh(basta_time_dt,\
                                                                 basta_height_bins*1.e-3,\
                                                                 ceil_backscatter_interp_linear[:,1:],\
                                                                 cmap=cmap,
                                                                 vmin=-8,vmax=-3)         
                


                # Colorbar
                dum_ticks = [-8,-7,-6,-5,-4,-3]
                interp_linear_cbar = fig.colorbar(interp_linear_plot,ticks=dum_ticks,pad=0.01,ax=ax_interp_linear)
                dumstr = '$log_{10}$($\\beta_{att}$)'
                interp_linear_cbar.ax.set_ylabel(dumstr,fontsize=Fontsize)
                interp_linear_cbar.ax.tick_params(labelsize=Fontsize)  

                ax_interp_linear.set_title('Linear Interpolation $\\beta_{att}$ \n 25-m x 12-sec resolution',fontsize=Fontsize*1.5,color='dimgrey')                             
            

            #-------------------------------------
            #-------------------------------------
            # Fog ID
            #-------------------------------------
            #-------------------------------------
            # For interpolated data, we want to come down from cloud base
            # and determine whether or not there is a decade decrease in
            # magnitude before reaching the lowest bin.
            
            fog_mask = np.zeros(np.shape(basta_time_dt))
            min_diff_arr_out = np.zeros(np.shape(basta_time_dt))
            for tt in range(len(basta_time_dt)):
                if ceil_cbh_1_interp[tt] > 0.:
                    height_id = np.where(basta_height == ceil_cbh_1_interp[tt])[0][0]
                    cbh_beta = ceil_backscatter_interp_nn[height_id,tt]
                    below_cbh_beta = ceil_backscatter_interp_nn[:height_id,tt]
                    min_below_cbh_beta = np.nanmin(below_cbh_beta)
                    dumdiff = cbh_beta - min_below_cbh_beta
                    min_diff_arr_out[tt] = dumdiff
                    if cbh_beta < -4.5:
                        continue
                    if np.isnan(dumdiff):
                        continue
                    elif (~np.isnan(dumdiff)) & (dumdiff >= 1.):
                        continue
                    elif (~np.isnan(dumdiff)) & (dumdiff < 1.):
                        fog_mask[tt] = 1
                else:
                    min_diff_arr_out[tt] = np.nan
                    
            # Shade fog periods
            # Fill in fog time periods with transparent blue
            fog_id = np.where(fog_mask == 1.)
            if np.size(fog_id) > 1.:
                fog_Objects,num_fog_objects = ndimage.label(fog_mask)
                for kk in range(num_fog_objects):
                    dumid = np.where(fog_Objects == kk+1)[0]
                    first_id = dumid[0]
                    last_id = dumid[-1]
                    ax_interp_nn.axvspan(basta_time_dt[first_id],\
                                basta_time_dt[last_id],color='navy',alpha=0.5)               
                   
            
            blue_patch = mpatches.Patch(color='navy',alpha=0.5,label='Fog')
            lgnd2 = ax_interp_nn.legend(handles=[blue_patch],\
                                fontsize=Fontsize*1.5,\
                                bbox_to_anchor=(0,1.2),\
                                ncol=1,loc='upper left',framealpha=0)                        
            
            
            ax_interp_nn.add_artist(lgnd)
            plt.subplots_adjust(hspace=0.3,wspace=0.1)
            plt.show()
            plt.close()
            
            
            
            
        #================================================
        # Interpolate backscatter to radar grid
        #================================================
        if interp_backscatter:
            basta_time_dt = np.array([toDatetime(basta_time_ts[dd]) for dd in range(len(basta_time_ts))])             
            start_time = datetime.datetime(basta_time_dt[0].year,basta_time_dt[0].month,basta_time_dt[0].day,0,0)
            end_time = start_time + datetime.timedelta(hours=24)          

            ceil_time_dt = np.array(ceil_time_dt)
            ceil_time_dt_orig = ceil_time_dt.copy()
            ceil_time_ts_orig = ceil_time_ts.copy()
            ceil_backscatter_orig = ceil_backscatter.copy()

            dumid = np.where( (ceil_time_ts >= basta_time_ts[0]) & (ceil_time_ts <= basta_time_ts[-1]))
            if np.size(dumid) > 0.:
                dumid = np.squeeze(dumid)
                ceil_time_ts = ceil_time_ts[dumid]
                ceil_time_dt = ceil_time_dt[dumid]
                ceil_backscatter = ceil_backscatter[dumid,:]        

            # Interpolated in log10 space (nearest neighbor)            
            x=ceil_time_ts = np.array([toTimestamp(ceil_time_dt[dd]) for dd in range(len(ceil_time_dt))])
            y=ceil_range
            #mask invalid values
            z = ceil_backscatter.copy()
            z = z.T
            xx, yy = np.meshgrid(x, y)
            basta_height_lim = basta_height[basta_height < np.max(ceil_range)]
            newy = basta_height_lim
            newx = basta_time_ts

            newX,newY = np.meshgrid(newx,newy)
            ceil_backscatter_interp = griddata((xx.ravel(), yy.ravel()), z.ravel(),(newX, newY),method='nearest',fill_value=np.nan)
            
            dumid = np.where( (basta_time_ts < ceil_time_ts_orig[0]) | (basta_time_ts > ceil_time_ts_orig[-1]) )
            if np.size(dumid) > 0.:
                dumid = np.squeeze(dumid)
                ceil_cbh_1_interp[dumid] = np.nan
                ceil_cbh_2_interp[dumid] = np.nan
                ceil_cbh_3_interp[dumid] = np.nan
                ceil_backscatter_interp[:,dumid] = np.nan
                ceil_detection_status_interp[dumid] = np.nan
                ceil_cbh_bin_relative_interp[dumid] = np.nan
            # Need to deal with missing times in between the start and end time of the radar
            # Let's loop through basta times and if the time is not within 12 seconds of a
            # radar profile, we'll nan it out
            for dum_tt in range(len(basta_time_dt)):
                dum_diff = np.abs(basta_time_ts[dum_tt] - ceil_time_ts)
                if np.min(dum_diff) > 12.:
                    ceil_backscatter_interp[:,dum_tt] = np.nan
        else:
                ceil_backscatter_interp = None
                basta_height_lim = None            

            
        
        arm_ceil_out_dict = {'cbh_1':ceil_cbh_1_interp,\
                    'cbh_2':ceil_cbh_2_interp,\
                    'cbh_3':ceil_cbh_3_interp,\
                    'detection_status':ceil_detection_status_interp,\
                    'backscatter':ceil_backscatter_interp,\
                    'native_cbh_1':ceil_cbh_1,\
                    'native_cbh_2':ceil_cbh_2,\
                    'native_cbh_3':ceil_cbh_3,\
                    'native_detection_status':ceil_detection_status,\
                    'native_backscatter':ceil_backscatter_orig,\
                    'native_time_dt':ceil_time_dt_orig,\
                    'native_time_ts':ceil_time_ts_orig,\
                    'native_range':ceil_range,\
                    'native_range_bounds':ceil_range_bounds,\
                    'interp_height':basta_height_lim,\
                    'cbh_bin_relative_interp':ceil_cbh_bin_relative_interp}
     
        return arm_ceil_present_flag,arm_ceil_out_dict

In [56]:
def process_aad_ceil(date,aad_ceil_dates_dt,aad_ceil_files,avg_bool,basta_height,basta_time_ts,interp_backscatter=False):

    tmpid = np.where(aad_ceil_dates_dt == date)
    if np.size(tmpid) == 0.:
        aad_ceil_present_flag = False
        aad_ceil_out_dict = None
        return aad_ceil_present_flag, aad_ceil_out_dict
    elif np.size(tmpid) > 0.:        
        aad_ceil_present_flag = True
        tmpid = tmpid[0][0]
        current_ceil_file = aad_ceil_files[tmpid]
        ncfile = xarray.open_dataset(current_ceil_file,decode_times=False)    
        ceil_dims = ncfile.dims
        ceil_time_ts = np.array(ncfile['time']).copy()
        ceil_cbh_1 = np.array(ncfile['cbh_1']).copy()
        ceil_cbh_2 = np.array(ncfile['cbh_2']).copy()
        ceil_cbh_3 = np.array(ncfile['cbh_3']).copy()
        ceil_detection_status = np.array(ncfile['detection_status']).copy()
        ceil_backscatter = np.array(ncfile['backscatter']).copy()
        ceil_height = np.array(ncfile['height']).copy()
        ceil_num_times = ceil_dims['time_dim']
        ceil_time_dt = np.array([toDatetime(ceil_time_ts[dd]) for dd in range(ceil_num_times)]  )  
        ncfile.close()

        #------------------------------------------
        # Interpolate ceilometer to radar time grid
        # using nearest neighbor interpolation. 
        # This method requires that the nearest
        # neighbor be within 15 seconds of the
        # radar time grid element.
        #------------------------------------------

        basta_bin_edges = np.arange(0,np.max(basta_height)+12.5+25.,25.)
        basta_time_dt = np.array([toDatetime(basta_time_ts[dd]) for dd in range(len(basta_time_ts))])

        ceil_cbh_1_interp = []
        ceil_cbh_2_interp = []
        ceil_cbh_3_interp = []
        ceil_detection_status_interp = []
        ceil_cbh_bin_relative_interp = []
        
        for ttt in range(len(basta_time_ts)):
            # if here, then good radar data exists
            # Now find the nearest in time ceilometer time step to the radar time step
            # If the ceilometer is more than 15 seconds away from the the radar time step,
            # then we will flag it as missing data (NaN)
            nearest_val,nearest_id = find_nearest(ceil_time_ts,basta_time_ts[ttt])
            time_diff = np.abs(nearest_val - basta_time_ts[ttt])
            target_time_diff = 16
            if time_diff <= target_time_diff:
                nearest_ceil_cbh_1 = ceil_cbh_1[nearest_id]
                nearest_ceil_cbh_2 = ceil_cbh_2[nearest_id]
                nearest_ceil_cbh_3 = ceil_cbh_3[nearest_id]
                nearest_ceil_detection_status = ceil_detection_status[nearest_id]
                ceil_detection_status_interp.append(nearest_ceil_detection_status)

                if np.isnan(nearest_ceil_detection_status):
                    #ceil_detection_status_interp.append(np.nan)
                    ceil_cbh_1_interp.append(np.nan)
                    ceil_cbh_2_interp.append(np.nan)
                    ceil_cbh_3_interp.append(np.nan)
                    ceil_cbh_bin_relative_interp.append(np.nan)
                    continue
                    
                elif (nearest_ceil_detection_status == 5.) or (nearest_ceil_detection_status == 0.):
                    ceil_cbh_1_interp.append(-999.)
                    ceil_cbh_2_interp.append(-999.)
                    ceil_cbh_3_interp.append(-999.)
                    ceil_cbh_bin_relative_interp.append(-999.)
                    continue
                    
                elif (nearest_ceil_detection_status == 4.):
                    ceil_cbh_1_interp.append(np.nan)
                    ceil_cbh_2_interp.append(np.nan)
                    ceil_cbh_3_interp.append(np.nan)
                    ceil_cbh_bin_relative_interp.append(np.nan)
                    continue                    
                    
                elif (nearest_ceil_detection_status == 1.) or (nearest_ceil_detection_status == 2.) or (nearest_ceil_detection_status == 3.):
                    pass
                else:
                    raise RuntimeError('Something went wrong')

                #if np.isnan(nearest_ceil_cbh_1):
                #    ceil_cbh_1_interp.append(np.nan)
                #    ceil_cbh_2_interp.append(np.nan)
                #    ceil_cbh_3_interp.append(np.nan)
                #    ceil_cbh_bin_relative_interp.append(np.nan)
                #    continue 
                    
                nearest_val,nearest_id = find_nearest(basta_bin_edges,nearest_ceil_cbh_1)
                if nearest_ceil_cbh_1 > 12000.:
                    ceil_cbh_1_interp.append(-999.)
                    ceil_cbh_2_interp.append(-999.)
                    ceil_cbh_3_interp.append(-999.)
                    ceil_cbh_bin_relative_interp.append(-999.)
                    ceil_detection_status_interp[ttt] = 0.
                else:
                    if nearest_ceil_cbh_1 == nearest_val:
                        bin_edges = basta_bin_edges[nearest_id-1:nearest_id+1]
                        midbin = (bin_edges[0]+bin_edges[1])/2.
                        ceil_cbh_1_interp.append(midbin)
                        ceil_cbh_bin_relative_interp.append(1.)
                    elif nearest_ceil_cbh_1 < nearest_val:
                        bin_edges = basta_bin_edges[nearest_id-1:nearest_id+1]
                        midbin = (bin_edges[0]+bin_edges[1])/2.
                        ceil_cbh_1_interp.append(midbin)
                        ceil_cbh_bin_relative_interp.append(0.)
                    elif nearest_ceil_cbh_1 > nearest_val:
                        bin_edges = basta_bin_edges[nearest_id:nearest_id+2]
                        midbin = (bin_edges[0]+bin_edges[1])/2.
                        ceil_cbh_1_interp.append(midbin)
                        ceil_cbh_bin_relative_interp.append(2.)

                    if np.isnan(nearest_ceil_cbh_2):
                        ceil_cbh_2_interp.append(np.nan)
                    else:    
                        nearest_val,nearest_id = find_nearest(basta_bin_edges,nearest_ceil_cbh_2)
                        if nearest_ceil_cbh_2 == nearest_val:
                            bin_edges = basta_bin_edges[nearest_id-1:nearest_id+1]
                            midbin = (bin_edges[0]+bin_edges[1])/2.
                            ceil_cbh_2_interp.append(midbin)
                        elif nearest_ceil_cbh_2 < nearest_val:
                            bin_edges = basta_bin_edges[nearest_id-1:nearest_id+1]
                            midbin = (bin_edges[0]+bin_edges[1])/2.
                            ceil_cbh_2_interp.append(midbin)
                        elif nearest_ceil_cbh_2 > nearest_val:
                            bin_edges = basta_bin_edges[nearest_id:nearest_id+2]
                            midbin = (bin_edges[0]+bin_edges[1])/2.
                            ceil_cbh_2_interp.append(midbin)

                    if np.isnan(nearest_ceil_cbh_3):
                        ceil_cbh_3_interp.append(np.nan)
                    else:                      
                        nearest_val,nearest_id = find_nearest(basta_bin_edges,nearest_ceil_cbh_3)
                        if nearest_ceil_cbh_3 == nearest_val:
                            bin_edges = basta_bin_edges[nearest_id-1:nearest_id+1]
                            midbin = (bin_edges[0]+bin_edges[1])/2.
                            ceil_cbh_3_interp.append(midbin)
                        elif nearest_ceil_cbh_3 < nearest_val:
                            bin_edges = basta_bin_edges[nearest_id-1:nearest_id+1]
                            midbin = (bin_edges[0]+bin_edges[1])/2.
                            ceil_cbh_3_interp.append(midbin)
                        elif nearest_ceil_cbh_3 > nearest_val:
                            bin_edges = basta_bin_edges[nearest_id:nearest_id+2]
                            midbin = (bin_edges[0]+bin_edges[1])/2.
                            ceil_cbh_3_interp.append(midbin)                    


                if np.isnan(ceil_cbh_1_interp[ttt]):
                    print(aaaa)
            else:
                #print('here')
                #print(time_diff,basta_time_dt[ttt],ceil_time_dt[nearest_id])
                ceil_cbh_1_interp.append(np.nan)
                ceil_cbh_2_interp.append(np.nan)
                ceil_cbh_3_interp.append(np.nan)
                ceil_detection_status_interp.append(np.nan)
                ceil_cbh_bin_relative_interp.append(np.nan)

        ceil_cbh_1_interp = np.array(ceil_cbh_1_interp)
        ceil_cbh_2_interp = np.array(ceil_cbh_2_interp)
        ceil_cbh_3_interp = np.array(ceil_cbh_3_interp)
        ceil_detection_status_interp = np.array(ceil_detection_status_interp)    
        ceil_cbh_bin_relative_interp = np.array(ceil_cbh_bin_relative_interp) 
            
        #dumid = np.where(ceil_height <= np.max(basta_height))
        #dumid = np.squeeze(dumid)
        #ceil_backscatter = ceil_backscatter[:,dumid]
        ceil_backscatter[ceil_backscatter == 0.] = np.nan
        
    

        
                
        #============================================
        # Plot to explore backscatter interpolation
        #============================================
        #if True: 
        if False:

            fig = plt.figure(figsize=(24,14))
            Fontsize=14
            dfmt = mdates.DateFormatter('%H:%M')
            ax_native = fig.add_subplot(211)
            ax_interp_nn = fig.add_subplot(212)
            
            basta_time_dt = np.array([toDatetime(basta_time_ts[dd]) for dd in range(len(basta_time_ts))])             
            start_time = datetime.datetime(basta_time_dt[0].year,basta_time_dt[0].month,basta_time_dt[0].day,0,0)
            #end_time = start_time + datetime.timedelta(days=1)           
            end_time = start_time + datetime.timedelta(hours=24)           
            
            axlist = [ax_native,ax_interp_nn]
            for ax in axlist:
                ax.tick_params(labelsize=Fontsize)
                ax.set_ylabel('Height [km]',fontsize=Fontsize)
                ax.set_xlabel('UTC Time [HH:MM]',fontsize=Fontsize)
                ax.xaxis.set_major_formatter(dfmt)
                ax.grid(which='both',c='dimgrey',ls='dotted',lw=1)
                ax.set_xlim(start_time,end_time)
                ax.set_ylim(0,1)
                
            cmap = matplotlib.cm.get_cmap("jet").copy()
            cmap.set_under('navy')
            cmap.set_bad('grey')

            # Native
            height_bins = np.arange(0,np.max(ceil_height),10)
            dumbin = np.array([height_bins[-1]+10])
            height_bins = np.concatenate((height_bins,dumbin))

            native_plot = ax_native.pcolormesh(ceil_time_dt,\
                                                             height_bins*1.e-3,\
                                                             ceil_backscatter[1:,:].T,\
                                                             cmap=cmap,
                                                             vmin=-8,vmax=-3)
            # Colorbar
            dum_ticks = [-8,-7,-6,-5,-4,-3]
            native_cbar = fig.colorbar(native_plot,ticks=dum_ticks,pad=0.01,ax=ax_native)
            dumstr = '$log_{10}$($\\beta_{att}$)'
            native_cbar.ax.set_ylabel(dumstr,fontsize=Fontsize)
            native_cbar.ax.tick_params(labelsize=Fontsize)  
            
            ax_native.set_title('Native $\\beta_{att}$ \n 10-m x 6-sec resolution',fontsize=Fontsize*1.5,color='dimgrey')
            ax_native.scatter(ceil_time_dt,ceil_cbh_1*1.e-3,s=2,c='black')
            
            # Fill in obscured time periods with transparent red
            id4 = np.where(ceil_detection_status == 4.)
            detection_mask = np.zeros(np.shape(ceil_detection_status))
            if np.size(id4) > 1.:
                id4 = np.squeeze(id4)
                detection_mask[id4] = 1
                detection_Objects,num_detection_objects = ndimage.label(detection_mask)
                for kk in range(num_detection_objects):
                    dumid = np.where(detection_Objects == kk+1)[0]
                    first_id = dumid[0]
                    last_id = dumid[-1]
                    ax_native.axvspan(ceil_time_dt[first_id],\
                                ceil_time_dt[last_id],color='red',alpha=0.5)               
            
            
            
            #ceil_time_dt = np.array(ceil_time_dt)
            ceil_time_dt_orig = ceil_time_dt.copy()
            ceil_time_ts_orig = ceil_time_ts.copy()
            
            dumid = np.where( (ceil_time_ts >= basta_time_ts[0]) & (ceil_time_ts <= basta_time_ts[-1]))
            if np.size(dumid) > 0.:
                dumid = np.squeeze(dumid)
                ceil_time_ts = ceil_time_ts[dumid]
                ceil_time_dt = ceil_time_dt[dumid]
                ceil_backscatter = ceil_backscatter[dumid,:]
                
                
            # Interpolated in log10 space (nearest neighbor)
            x=ceil_time_ts = np.array([toTimestamp(ceil_time_dt[dd]) for dd in range(len(ceil_time_dt))])
            y=ceil_height
            #mask invalid values
            z = ceil_backscatter.copy()
            z = z.T
            xx, yy = np.meshgrid(x, y)
            #basta_height_lim = basta_height[basta_height < np.max(ceil_range)]
            #newy = basta_height_lim
            newy = basta_height
            newx = basta_time_ts
            newX,newY = np.meshgrid(newx,newy)
            ceil_backscatter_interp_nn = griddata((xx.ravel(), yy.ravel()), z.ravel(),(newX, newY),method='nearest',fill_value=np.nan)
            
            dumid = np.where( (basta_time_ts < ceil_time_ts_orig[0]) | (basta_time_ts > ceil_time_ts_orig[-1]) )
            if np.size(dumid) > 0.:
                dumid = np.squeeze(dumid)
                ceil_cbh_1_interp[dumid] = np.nan
                ceil_cbh_2_interp[dumid] = np.nan
                ceil_cbh_3_interp[dumid] = np.nan
                ceil_backscatter_interp_nn[:,dumid] = np.nan
                ceil_detection_status_interp[dumid] = np.nan
                ceil_cbh_bin_relative_interp[dumid] = np.nan
                
            # Need to deal with missing times in between the start and end time of the radar
            # Let's loop through basta times and if the time is not within 12 seconds of a
            # radar profile, we'll nan it out

            for dum_tt in range(len(basta_time_dt)):
                dum_diff = np.abs(basta_time_ts[dum_tt] - ceil_time_ts)
                if np.min(dum_diff) > 12.:
                    ceil_backscatter_interp_nn[:,dum_tt] = np.nan
            
                
            basta_height_bins = np.arange(0,np.max(basta_height),25)
            dumbins = np.array([np.max(basta_height)+12.5])
            basta_height_bins = np.concatenate((basta_height_bins,dumbins))
            interp_nn_plot = ax_interp_nn.pcolormesh(basta_time_dt,\
                                                             basta_height_bins*1.e-3,\
                                                             ceil_backscatter_interp_nn[:,1:],\
                                                             cmap=cmap,
                                                             vmin=-8,vmax=-3)            
            
            # Colorbar
            dum_ticks = [-8,-7,-6,-5,-4,-3]
            interp_nn_cbar = fig.colorbar(interp_nn_plot,ticks=dum_ticks,pad=0.01,ax=ax_interp_nn)
            dumstr = '$log_{10}$($\\beta_{att}$)'
            interp_nn_cbar.ax.set_ylabel(dumstr,fontsize=Fontsize)
            interp_nn_cbar.ax.tick_params(labelsize=Fontsize)  
            
            ax_interp_nn.set_title('Nearest Neighbor Interpolation $\\beta_{att}$ \n 25-m x 12-sec resolution',fontsize=Fontsize*1.5,color='dimgrey')            
            # Interpolated CBH
            ax_interp_nn.scatter(basta_time_dt,ceil_cbh_1_interp*1.e-3,s=2,c='black')

            
            # Fill in obscured time periods with transparent red
            id4 = np.where(ceil_detection_status_interp == 4.)
            detection_mask = np.zeros(np.shape(ceil_detection_status_interp))
            if np.size(id4) > 1.:
                id4 = np.squeeze(id4)
                detection_mask[id4] = 1
                detection_Objects,num_detection_objects = ndimage.label(detection_mask)
                for kk in range(num_detection_objects):
                    dumid = np.where(detection_Objects == kk+1)[0]
                    first_id = dumid[0]
                    last_id = dumid[-1]
                    ax_interp_nn.axvspan(basta_time_dt[first_id],\
                                basta_time_dt[last_id],color='red',alpha=0.5)               
            
            red_patch = mpatches.Patch(color='red',alpha=0.5,label='CEIL obscured')
            lgnd = ax_interp_nn.legend(handles=[red_patch],\
                                fontsize=Fontsize*1.5,\
                                bbox_to_anchor=(1,1.2),\
                                ncol=1,loc='upper right',framealpha=0)
            

            #-------------------------------------
            #-------------------------------------
            # Fog ID
            #-------------------------------------
            #-------------------------------------
            # For interpolated data, we want to come down from cloud base
            # and determine whether or not there is a decade decrease in
            # magnitude before reaching the lowest bin.
            
            fog_mask = np.zeros(np.shape(basta_time_dt))
            min_diff_arr_out = np.zeros(np.shape(basta_time_dt))
            for tt in range(len(basta_time_dt)):
                if ceil_cbh_1_interp[tt] > 0.:
                    height_id = np.where(basta_height == ceil_cbh_1_interp[tt])#[0][0]
                    #height_id = np.squeeze(height_id)
                    #height_id = height_id[0]
                    #print(np.shape(height_id))
                    continue
                    cbh_beta = ceil_backscatter_interp_nn[height_id,tt]
                    below_cbh_beta = ceil_backscatter_interp_nn[:height_id,tt]
                    min_below_cbh_beta = np.nanmin(below_cbh_beta)
                    dumdiff = cbh_beta - min_below_cbh_beta
                    min_diff_arr_out[tt] = dumdiff
                    if cbh_beta < -4.5:
                        continue
                    if np.isnan(dumdiff):
                        continue
                    elif (~np.isnan(dumdiff)) & (dumdiff >= 1.):
                        continue
                    elif (~np.isnan(dumdiff)) & (dumdiff < 1.):
                        fog_mask[tt] = 1
                else:
                    min_diff_arr_out[tt] = np.nan
            #print(aaaa)
            # Shade fog periods
            # Fill in fog time periods with transparent blue
            fog_id = np.where(fog_mask == 1.)
            if np.size(fog_id) > 1.:
                fog_Objects,num_fog_objects = ndimage.label(fog_mask)
                for kk in range(num_fog_objects):
                    dumid = np.where(fog_Objects == kk+1)[0]
                    first_id = dumid[0]
                    last_id = dumid[-1]
                    ax_interp_nn.axvspan(basta_time_dt[first_id],\
                                basta_time_dt[last_id],color='navy',alpha=0.5)               
                   
            
            blue_patch = mpatches.Patch(color='navy',alpha=0.5,label='Fog')
            lgnd2 = ax_interp_nn.legend(handles=[blue_patch],\
                                fontsize=Fontsize*1.5,\
                                bbox_to_anchor=(0,1.2),\
                                ncol=1,loc='upper left',framealpha=0)                        
            
            
            ax_interp_nn.add_artist(lgnd)
            plt.subplots_adjust(hspace=0.3,wspace=0.1)
            plt.show()
            plt.close()

          
            
        #================================================
        # Interpolate backscatter to radar grid
        #================================================            
        if interp_backscatter:
            basta_time_dt = np.array([toDatetime(basta_time_ts[dd]) for dd in range(len(basta_time_ts))])             
            start_time = datetime.datetime(basta_time_dt[0].year,basta_time_dt[0].month,basta_time_dt[0].day,0,0)
            end_time = start_time + datetime.timedelta(hours=24)          

            ceil_time_dt = np.array(ceil_time_dt)
            ceil_time_dt_orig = ceil_time_dt.copy()
            ceil_time_ts_orig = ceil_time_ts.copy()
            ceil_backscatter_orig = ceil_backscatter.copy()

            dumid = np.where( (ceil_time_ts >= basta_time_ts[0]) & (ceil_time_ts <= basta_time_ts[-1]))
            if np.size(dumid) > 0.:
                dumid = np.squeeze(dumid)
                ceil_time_ts = ceil_time_ts[dumid]
                ceil_time_dt = ceil_time_dt[dumid]
                ceil_backscatter = ceil_backscatter[dumid,:]        

            # Interpolated in log10 space (nearest neighbor)
            x=ceil_time_ts = np.array([toTimestamp(ceil_time_dt[dd]) for dd in range(len(ceil_time_dt))])
            y=ceil_height
            #mask invalid values
            z = ceil_backscatter.copy()
            z = z.T
            xx, yy = np.meshgrid(x, y)
            #basta_height_lim = basta_height[basta_height < np.max(ceil_range)]
            #newy = basta_height_lim
            newy = basta_height
            newx = basta_time_ts
            newX,newY = np.meshgrid(newx,newy)
            ceil_backscatter_interp = griddata((xx.ravel(), yy.ravel()), z.ravel(),(newX, newY),method='nearest',fill_value=np.nan)

            dumid = np.where( (basta_time_ts < ceil_time_ts_orig[0]) | (basta_time_ts > ceil_time_ts_orig[-1]) )
            if np.size(dumid) > 0.:
                dumid = np.squeeze(dumid)
                ceil_cbh_1_interp[dumid] = np.nan
                ceil_cbh_2_interp[dumid] = np.nan
                ceil_cbh_3_interp[dumid] = np.nan
                ceil_backscatter_interp[:,dumid] = np.nan
                ceil_detection_status_interp[dumid] = np.nan
                ceil_cbh_bin_relative_interp[dumid] = np.nan

            # Need to deal with missing times in between the start and end time of the radar
            # Let's loop through basta times and if the time is not within 16 seconds of a
            # radar profile, we'll nan it out
            for dum_tt in range(len(basta_time_dt)):
                dum_diff = np.abs(basta_time_ts[dum_tt] - ceil_time_ts)
                if np.min(dum_diff) > 16.:
                    ceil_backscatter_interp[:,dum_tt] = np.nan
        else:
            ceil_backscatter_interp = None
            basta_height_lim = None            
                  
        
        aad_ceil_out_dict = {'cbh_1':ceil_cbh_1_interp,\
                    'cbh_2':ceil_cbh_2_interp,\
                    'cbh_3':ceil_cbh_3_interp,\
                    'detection_status':ceil_detection_status_interp,\
                    'backscatter':ceil_backscatter_interp,\
                    'native_cbh_1':ceil_cbh_1,\
                    'native_cbh_2':ceil_cbh_2,\
                    'native_cbh_3':ceil_cbh_3,\
                    'native_detection_status':ceil_detection_status,\
                    'native_backscatter':ceil_backscatter_orig,\
                    'native_time_dt':ceil_time_dt_orig,\
                    'native_time_ts':ceil_time_ts_orig,\
                    'native_range':ceil_height,\
                    'interp_height':basta_height,\
                    'cbh_bin_relative_interp':ceil_cbh_bin_relative_interp,\
                   }
     
        return aad_ceil_present_flag,aad_ceil_out_dict

In [57]:
def merge_ceil(aad_ceil_dict,arm_ceil_dict,basta_time_dt):

    arm_cbh_1 = arm_ceil_dict['cbh_1']
    arm_cbh_2 = arm_ceil_dict['cbh_2']
    arm_cbh_3 = arm_ceil_dict['cbh_3']
    arm_detection_status = arm_ceil_dict['detection_status']
    arm_cbh_bin_relative_interp = arm_ceil_dict['cbh_bin_relative_interp']

    aad_cbh_1 = aad_ceil_dict['cbh_1']
    aad_cbh_2 = aad_ceil_dict['cbh_2']
    aad_cbh_3 = aad_ceil_dict['cbh_3']
    aad_detection_status = aad_ceil_dict['detection_status']
    aad_cbh_bin_relative_interp = aad_ceil_dict['cbh_bin_relative_interp']    
    
    #merge_cbh_1 = np.zeros(np.shape(basta_time_dt))
    #merge_cbh_2 = np.zeros(np.shape(basta_time_dt))
    #merge_cbh_3 = np.zeros(np.shape(basta_time_dt))
    #merge_detection_status = np.zeros(np.shape(basta_time_dt))
    #merge_cbh_bin_relative_interp = np.zeros(np.shape(basta_time_dt))
    #merge_source_ceil = np.zeros(np.shape(basta_time_dt))
    
    merge_cbh_1 = arm_cbh_1.copy()
    merge_cbh_2 = arm_cbh_2.copy()
    merge_cbh_3 = arm_cbh_3.copy()
    merge_detection_status = arm_detection_status.copy()
    merge_cbh_bin_relative_interp = arm_cbh_bin_relative_interp.copy()
    merge_source_ceil = np.zeros(np.shape(basta_time_dt))
    merge_source_ceil[:] = 1
    
    # Anything with ceil_detection_status == NaN means that the ceilometer
    # did not have a reading within 16 seconds or that the ceilometer value was
    # generally bad. 
    #
    # If one ceilometer exists at this time step but the other doesn't, then the merge
    # ceilometer will use the ceilometer that DOES exist
    dumid_nan = np.where(np.isnan(arm_detection_status) & np.isnan(aad_detection_status))
    dumid_aad = np.where( np.isnan(arm_detection_status) & ~np.isnan(aad_detection_status) )
    dumid_arm = np.where( ~np.isnan(arm_detection_status) & np.isnan(aad_detection_status) )
    dumid_id4 = np.where( (arm_detection_status == 4.) & (aad_detection_status != 4.) & ~np.isnan(aad_detection_status) )
    #print('nan:',np.size(dumid_nan))
    #print('aad:',np.size(dumid_aad))
    #print('arm:',np.size(dumid_arm))
    #print('id4:',np.size(dumid_id4))
    
    # ARM Ceilometer is obscured but AAD Ceilometer is not
    if np.size(dumid_id4) > 0.:
        dumid_id4 = np.squeeze(dumid_id4)
        merge_cbh_bin_relative_interp[dumid_id4] = aad_cbh_bin_relative_interp[dumid_id4]
        merge_cbh_1[dumid_id4] = aad_cbh_1[dumid_id4]
        merge_cbh_2[dumid_id4] = aad_cbh_2[dumid_id4]
        merge_cbh_3[dumid_id4] = aad_cbh_3[dumid_id4]
        merge_detection_status[dumid_id4] = aad_detection_status[dumid_id4]
        merge_source_ceil[dumid_id4] = 2
    
    # No ceilometer at all
    if np.size(dumid_nan) > 0.:
        dumid_nan = np.squeeze(dumid_nan)
        merge_cbh_bin_relative_interp[dumid_nan] = np.nan
        merge_cbh_1[dumid_nan] = np.nan
        merge_cbh_2[dumid_nan] = np.nan
        merge_cbh_3[dumid_nan] = np.nan
        merge_detection_status[dumid_nan] = np.nan
        merge_source_ceil[dumid_nan] = np.nan
        
    #if np.size(dumid_999) > 0.:
    #    dumid_999 = np.squeeze(dumid_999)
    #    merge_cbh_bin_relative_interp[dumid_999] = np.nan
    #    merge_detection_status[dumid_999] = np.nan
    #    merge_cbh_1[dumid_999] = np.nan
    #    merge_cbh_2[dumid_999] = np.nan
    #    merge_cbh_3[dumid_999] = np.nan
    #    merge_source_ceil[dumid_999] = np.nan
    
    #if False:
    if True:
        if np.size(dumid_aad) > 0.:
            dumid_aad = np.squeeze(dumid_aad)
            merge_cbh_bin_relative_interp[dumid_aad] = aad_cbh_bin_relative_interp[dumid_aad]
            merge_detection_status[dumid_aad] = aad_detection_status[dumid_aad]
            merge_cbh_1[dumid_aad] = aad_cbh_1[dumid_aad]
            merge_cbh_2[dumid_aad] = aad_cbh_2[dumid_aad]
            merge_cbh_3[dumid_aad] = aad_cbh_3[dumid_aad]
            merge_source_ceil[dumid_aad] = 2
    if False:
        if np.size(dumid_arm) > 0.:
            dumid_arm = np.squeeze(dumid_arm)
            merge_cbh_bin_relative_interp[dumid_arm] = arm_cbh_bin_relative_interp[dumid_arm]
            merge_detection_status[dumid_arm] = arm_detection_status[dumid_arm]
            merge_cbh_1[dumid_arm] = arm_cbh_1[dumid_arm]
            merge_cbh_2[dumid_arm] = arm_cbh_2[dumid_arm]
            merge_cbh_3[dumid_arm] = arm_cbh_3[dumid_arm]    
            merge_source_ceil[dumid_arm] = 1
        

    merge_ceil_out_dict = {'detection_status':merge_detection_status,\
                           'cbh_bin_relative_interp':merge_cbh_bin_relative_interp,\
                           'cbh_1':merge_cbh_1,\
                           'cbh_2':merge_cbh_2,\
                           'cbh_3':merge_cbh_3,\
                           'source_ceil':merge_source_ceil,\
                          }
    
    return merge_ceil_out_dict

In [58]:
#==================================================
# Main function to merge instruments
#
# Inputs: date -- from basta_dates_dt
#         avg_bool -- boolean to perform averaging
#==================================================
def merge_instruments(date,basta_files,basta_dates_dt,avg_bool,arm_ceil_dates_dt,arm_files,aad_ceil_dates_dt,aad_ceil_files,sonde_dates_dt,sonde_files):
    print('-------------------------------------------------')
    print('-------------------------------------------------')
    print('Date: ',date[0].strftime('%Y/%m/%d'))
    print('-------------------------------------------------')
    print('-------------------------------------------------')
    
    # Dictionary that will hold all merged data, separated by intrument
    merged_dict = {}
    
    # Process BASTA data
    start_time = time.time()
    print('Processing BASTA data...')
    basta_present_flag,basta_out_dict = process_basta(date,basta_dates_dt,basta_files,avg_bool)
    if not basta_present_flag:
        print('Bad radar data on this date. Skipping')
        merged_dict = None
        return basta_present_flag,merged_dict
    else:
        pass
    merged_dict['basta'] = basta_out_dict
    end_time = time.time()
    elapsed_time = end_time - start_time
    print('Completed processing BASTA data. Time: {} seconds'.format(elapsed_time))
    if not basta_present_flag:
        print('No valid radar data on this date. Skipping')
        return basta_present_flag
    

    # Process AAD CEIL data
    start_time = time.time()
    print('Processing Univ. of Canterbury (AAD) CEIL data...')
    aad_ceil_present_flag,aad_ceil_out_dict = process_aad_ceil(date,aad_ceil_dates_dt,aad_ceil_files,avg_bool,basta_out_dict['height'],basta_out_dict['time_ts'],interp_backscatter=True)
    if aad_ceil_present_flag:
        merged_dict['aad_ceil'] = aad_ceil_out_dict
    end_time = time.time()
    elapsed_time = end_time - start_time
    print('Completed processing Univ. of Canterbury (AAD) CEIL data. Time: {} seconds'.format(elapsed_time))   
    
    
    
    # Process ARM CEIL data
    start_time = time.time()
    print('Processing ARM CEIL data...')
    arm_ceil_present_flag,arm_ceil_out_dict = process_arm_ceil(date,arm_ceil_dates_dt,arm_ceil_files,avg_bool,basta_out_dict['height'],basta_out_dict['time_ts'],interp_backscatter=True)
    if arm_ceil_present_flag:
        merged_dict['arm_ceil'] = arm_ceil_out_dict
    end_time = time.time()
    elapsed_time = end_time - start_time
    print('Completed processing ARM CEIL data. Time: {} seconds'.format(elapsed_time))




    
    # Process Sounding data
    start_time = time.time()
    print('Processing native sonde data...')
    sonde_present_flag,native_sonde_out_dict = process_native_sondes(date,sonde_dates_dt,sonde_times_dt,sonde_files,cluster_ids,cluster_times_dt)
    if sonde_present_flag:
        merged_dict['native_sonde'] = native_sonde_out_dict
    end_time = time.time()
    elapsed_time = end_time - start_time
    print('Completed processing native sonde data. Time: {} seconds'.format(elapsed_time))  
        
    # Interp Soundings
    start_time = time.time()
    print('Interpolating sonde data...')
    interp_sonde_present_flag, interp_sonde_out_dict = interp_sondes(in_date,sonde_dates_dt,sonde_times_dt,sonde_files,merged_dict['basta']['time_dt'],merged_dict['basta']['height'],cluster_ids,cluster_times_dt)
    if interp_sonde_present_flag:
        merged_dict['interp_sonde'] = interp_sonde_out_dict
    end_time = time.time()
    elapsed_time = end_time - start_time
    print('Completed interpolation of sonde data. Time: {} seconds'.format(elapsed_time))  
        
    # Process SFC Met
    start_time = time.time()
    print('Processing sfc met data...')
    sfc_met_present_flag,sfc_met_out_dict = process_sfc_met(date,sfc_dates_dt,sfc_files,avg_bool,basta_out_dict['time_ts'])
    if sfc_met_present_flag:
        merged_dict['sfc_met'] = sfc_met_out_dict
    end_time = time.time()
    elapsed_time = end_time - start_time
    print('Completed processing sfc met data. Time: {} seconds'.format(elapsed_time))         
        
    # Process satellite
    start_time = time.time()
    print('Processing satellite data...')
    sat_present_flag,sat_out_dict = process_sat(date,sat_dates_dt,sat_files)

    if sat_present_flag:
        merged_dict['sat'] = sat_out_dict
    end_time = time.time()
    elapsed_time = end_time - start_time
    print('Completed processing satellite data. Time: {} seconds'.format(elapsed_time))   
        
    # Process PIRAT
    start_time = time.time()
    print('Processing PIRAT data...')
    dis_present_flag,dis_out_dict = process_dis(date,dis_dates_dt,dis_files,avg_bool,basta_out_dict['time_ts'])
    if dis_present_flag:
        merged_dict['dis'] = dis_out_dict
    end_time = time.time()
    elapsed_time = end_time - start_time
    print('Completed processing PIRAT data. Time: {} seconds'.format(elapsed_time))  
        
    # Process Optics
    start_time = time.time()
    print('Processing optics data...')
    optics_present_flag,optics_out_dict = process_optics(date,optics_dates_dt,optics_files,avg_bool,basta_out_dict['time_ts'])
    if optics_present_flag:
        merged_dict['optics'] = optics_out_dict
    end_time = time.time()
    elapsed_time = end_time - start_time
    print('Completed processing optics data. Time: {} seconds'.format(elapsed_time))  
    
    

    # Merge ceilometers (prioritize AAD ceilometer)
    if not aad_ceil_present_flag and not arm_ceil_present_flag:
        # If neither ceilometer is present
        merge_ceil_out_dict = None
        merge_ceil_present_flag = False
    elif aad_ceil_present_flag and not arm_ceil_present_flag:
        # if AAD is only ceilometer present
        merge_ceil_out_dict = {}
        merge_ceil_out_dict['cbh_1'] = merged_dict['aad_ceil']['cbh_1']
        merge_ceil_out_dict['cbh_2'] = merged_dict['aad_ceil']['cbh_2']
        merge_ceil_out_dict['cbh_3'] = merged_dict['aad_ceil']['cbh_3']
        merge_ceil_out_dict['detection_status'] = merged_dict['aad_ceil']['detection_status']
        merge_ceil_out_dict['cbh_bin_relative_interp'] = merged_dict['aad_ceil']['cbh_bin_relative_interp']
        dum = np.zeros(np.shape(merged_dict['aad_ceil']['detection_status']))
        dum[:] = 2
        merge_ceil_out_dict['source_ceil'] = dum
        merge_ceil_present_flag = True
        merged_dict['merge_ceil'] = merge_ceil_out_dict
    elif not aad_ceil_present_flag and arm_ceil_present_flag:
        # if ARM is only ceilometer present
        merge_ceil_out_dict = {}
        merge_ceil_out_dict['cbh_1'] = merged_dict['arm_ceil']['cbh_1']
        merge_ceil_out_dict['cbh_2'] = merged_dict['arm_ceil']['cbh_2']
        merge_ceil_out_dict['cbh_3'] = merged_dict['arm_ceil']['cbh_3']
        merge_ceil_out_dict['detection_status'] = merged_dict['arm_ceil']['detection_status']
        merge_ceil_out_dict['cbh_bin_relative_interp'] = merged_dict['arm_ceil']['cbh_bin_relative_interp']
        dum = np.zeros(np.shape(merged_dict['arm_ceil']['detection_status']))
        dum[:] = 1
        merge_ceil_out_dict['source_ceil'] = dum
        merge_ceil_present_flag = True
        merged_dict['merge_ceil'] = merge_ceil_out_dict
    elif aad_ceil_present_flag and arm_ceil_present_flag:
        # if both ceilometers present
        #merge_ceil_out_dict = {}
        #merge_ceil_out_dict['cbh_1'] = merged_dict['aad_ceil']['cbh_1']
        #merge_ceil_out_dict['cbh_2'] = merged_dict['aad_ceil']['cbh_2']
        #merge_ceil_out_dict['cbh_3'] = merged_dict['aad_ceil']['cbh_3']
        #merge_ceil_out_dict['detction_status'] = merged_dict['aad_ceil']['detection_status']
        merge_ceil_out_dict = merge_ceil(merged_dict['aad_ceil'],merged_dict['arm_ceil'],merged_dict['basta']['time_dt'])
        merge_ceil_present_flag = True
        merged_dict['merge_ceil'] = merge_ceil_out_dict
        #merge_ceil_out_dict = None
        #merged_dict['merge_ceil'] = merge_ceil_out_dict
        
    # Dataset Flags
    merged_dict['dataset_flags'] = {'basta':basta_present_flag,\
                                    'arm_ceil':arm_ceil_present_flag,\
                                    'aad_ceil':aad_ceil_present_flag,\
                                    'sfc_met':sfc_met_present_flag,\
                                    'dis':dis_present_flag,\
                                    'sat':sat_present_flag,\
                                    'merge_ceil':merge_ceil_present_flag,\
                                    'interp_sonde':interp_sonde_present_flag,\
                                    'optics':optics_present_flag,\
                                    'native_sonde':sonde_present_flag}
       
    

    
    
    return basta_present_flag,merged_dict

# Main loop to merge instruments

In [60]:
# Note: avg_bool was a placeholder that was meant to temporally average data, but this feature was never implemented.
for date in basta_dates_dt:
    in_date = [date]
    basta_present_flag,merged_dict = merge_instruments(in_date,basta_files,basta_dates_dt,False,arm_ceil_dates_dt,arm_ceil_files,aad_ceil_dates_dt,aad_ceil_files,sonde_dates_dt,sonde_files)
    if basta_present_flag:
        save_path = '/mnt/raid/mwstanfo/micre/merged_instrument_files/'
        dum_time_str = in_date[0].strftime('%Y%m%d')
        out_pkl_file = save_path+'merged_instruments_{}.p'.format(dum_time_str)
        pickle.dump(merged_dict,open(out_pkl_file,"wb"))
        #print(aaaa)


-------------------------------------------------
-------------------------------------------------
Date:  2017/01/27
-------------------------------------------------
-------------------------------------------------
Processing BASTA data...
Completed processing BASTA data. Time: 0.7709634304046631 seconds
Processing Univ. of Canterbury (AAD) CEIL data...
Completed processing Univ. of Canterbury (AAD) CEIL data. Time: 42.07610487937927 seconds
Processing ARM CEIL data...
Completed processing ARM CEIL data. Time: 0.00011372566223144531 seconds
Processing native sonde data...
Completed processing native sonde data. Time: 0.2898678779602051 seconds
Interpolating sonde data...
# of current soundings: 3
Completed interpolation of sonde data. Time: 0.3427541255950928 seconds
Processing sfc met data...
Completed processing sfc met data. Time: 0.07286572456359863 seconds
Processing satellite data...
Completed processing satellite data. Time: 2.4031264781951904 seconds
Processing PIRAT data...

In [ ]:
#target_date = datetime.datetime(2016,5,22)
#dumid = np.where(basta_dates_dt == target_date)
#in_date = basta_dates_dt[dumid[0]]
#basta_present_flag,merged_dict = merge_instruments(in_date,basta_files,basta_dates_dt,False,arm_ceil_dates_dt,arm_ceil_files,aad_ceil_dates_dt,aad_ceil_files,sonde_dates_dt,sonde_files)

# Everything below is diagnostics

In [ ]:
dum_cbh = merged_dict['arm_ceil']['cbh_1']
dum_ds = merged_dict['arm_ceil']['detection_status']
dumid = np.where(dum_ds == 0.)
print('Detection Status 0:',np.unique(dum_cbh[dumid]))
dumid = np.where(dum_ds == 1.)
print('Detection Status 1:',np.unique(dum_cbh[dumid]))
dumid = np.where(dum_ds == 2.)
print('Detection Status 2:',np.unique(dum_cbh[dumid]))
dumid = np.where(dum_ds == 3.)
print('Detection Status 3:',np.unique(dum_cbh[dumid]))
dumid = np.where(dum_ds == 4.)
print('Detection Status 4:',np.unique(dum_cbh[dumid]))
dumid = np.where(dum_ds == 5.)
print('Detection Status 5:',np.unique(dum_cbh[dumid]))
dumid = np.where(np.isnan(dum_ds))
print('Detection Status NaN:',np.unique(dum_cbh[dumid]))

In [ ]:
dum_cbh = merged_dict['aad_ceil']['cbh_1']
dum_ds = merged_dict['aad_ceil']['detection_status']
dumid = np.where(dum_ds == 0.)
print('Detection Status 0:',np.unique(dum_cbh[dumid]))
dumid = np.where(dum_ds == 1.)
print('Detection Status 1:',np.unique(dum_cbh[dumid]))
dumid = np.where(dum_ds == 2.)
print('Detection Status 2:',np.unique(dum_cbh[dumid]))
dumid = np.where(dum_ds == 3.)
print('Detection Status 3:',np.unique(dum_cbh[dumid]))
dumid = np.where(dum_ds == 4.)
print('Detection Status 4:',np.unique(dum_cbh[dumid]))
dumid = np.where(dum_ds == 5.)
print('Detection Status 5:',np.unique(dum_cbh[dumid]))
dumid = np.where(np.isnan(dum_ds))
print('Detection Status NaN:',np.unique(dum_cbh[dumid]))

In [ ]:
dum_cbh = merged_dict['merge_ceil']['cbh_1']
dum_ds = merged_dict['merge_ceil']['detection_status']
dumid = np.where(dum_ds == 0.)
print('Detection Status 0:',np.unique(dum_cbh[dumid]))
dumid = np.where(dum_ds == 1.)
print('Detection Status 1:',np.unique(dum_cbh[dumid]))
dumid = np.where(dum_ds == 2.)
print('Detection Status 2:',np.unique(dum_cbh[dumid]))
dumid = np.where(dum_ds == 3.)
print('Detection Status 3:',np.unique(dum_cbh[dumid]))
dumid = np.where(dum_ds == 4.)
print('Detection Status 4:',np.unique(dum_cbh[dumid]))
dumid = np.where(dum_ds == 5.)
print('Detection Status 5:',np.unique(dum_cbh[dumid]))
dumid = np.where(np.isnan(dum_ds))
print('Detection Status NaN:',np.unique(dum_cbh[dumid]))

In [ ]:
#----------------------------------------------------
# Plot to check interpolation of ARM CEIL beta_att
# to radar time-height grid
#----------------------------------------------------
fig = plt.figure(figsize=(14,12))
ax1=fig.add_subplot(311)
ax2=fig.add_subplot(312)
ax3=fig.add_subplot(313)
axlist = [ax1,ax2,ax3]
Fontsize=14
dfmt = mdates.DateFormatter('%H:%M')


# Plot Reflectivity
dum_ref = merged_dict['basta']['ref'].copy()
dum_ref[(dum_ref > -999.) & (dum_ref < -50.)] = -50.
dum_ref[dum_ref > 20.] = 20.
cmap = matplotlib.cm.get_cmap("nipy_spectral").copy()
cmap.set_under('w')
cmap.set_bad('grey')
basta_bin_edges = np.arange(0,np.max(merged_dict['basta']['height'])+12.5+25.,25.)
dumid = np.where(merged_dict['basta']['height'] <= dum_y_lim*1.e3)
dumid = np.squeeze(dumid)

ref_plot = ax1.pcolormesh(merged_dict['basta']['time_dt'],\
                            merged_dict['basta']['height'][dumid]/1.e3,\
                            dum_ref[dumid,:],\
                            cmap=cmap,\
                            vmin=-50.1,vmax=20.1)            
ref_cbar = fig.colorbar(ref_plot,ax=ax1,pad=0.005)
ref_cbar.ax.set_ylabel('Reflectivity [dBZ]',fontsize=Fontsize)
ref_cbar.ax.tick_params(labelsize=Fontsize)
if np.all(merged_dict['arm_ceil']['cbh_1'] != np.nan):
    ax1.scatter(merged_dict['basta']['time_dt'],merged_dict['arm_ceil']['cbh_1']*1.e-3,s=2,c='k',label='CBH1')
if np.all(merged_dict['arm_ceil']['cbh_2'] != np.nan):
    ax1.scatter(merged_dict['basta']['time_dt'],merged_dict['arm_ceil']['cbh_2']*1.e-3,s=2,c='magenta',label='CBH2')
if np.all(merged_dict['arm_ceil']['cbh_3'] != np.nan):
    ax1.scatter(merged_dict['basta']['time_dt'],merged_dict['arm_ceil']['cbh_3']*1.e-3,s=2,c='cyan',label='CBH3')
ax1.legend(fontsize=Fontsize,loc='upper left',ncol=1)

# Plot Attenuated Backscatter
dum_arm_ceil_backscatter = merged_dict['arm_ceil']['native_backscatter'].copy()
cmap = matplotlib.cm.get_cmap("jet").copy()
cmap.set_under('navy')
cmap.set_bad('grey')
dum_arm_ceil_backscatter[dum_arm_ceil_backscatter < -8.] = -8.
dum_arm_ceil_backscatter[dum_arm_ceil_backscatter > -3.] = -3.
dumid = np.where(merged_dict['aad_ceil']['native_range'] <= dum_y_lim*1.e3)
dumid = np.squeeze(dumid)
ceil_backscatter_plot = ax2.pcolormesh(merged_dict['arm_ceil']['native_time_dt'],\
                                     merged_dict['arm_ceil']['native_range'][dumid]*1.e-3,\
                                     np.transpose(dum_arm_ceil_backscatter[:,dumid]),\
                                     cmap=cmap,
                                     vmin=-8,vmax=-3)
dum_ticks = [-8,-7,-6,-5,-4,-3]
ceil_backscatter_cbar = fig.colorbar(ceil_backscatter_plot,ax=ax2,pad=0.005,ticks=dum_ticks)
ceil_backscatter_cbar.ax.set_ylabel('$log_{10}$($\\beta_{att}$) [sr$^{-1}$ m$^{-1}$]',fontsize=Fontsize)
ceil_backscatter_cbar.ax.tick_params(labelsize=Fontsize)


if np.all(merged_dict['arm_ceil']['native_cbh_1'] != np.nan):
    ax2.scatter(merged_dict['arm_ceil']['native_time_dt'],merged_dict['arm_ceil']['native_cbh_1']*1.e-3,s=2,c='k',label='CBH1')
if np.all(merged_dict['arm_ceil']['native_cbh_2'] != np.nan):
    ax2.scatter(merged_dict['arm_ceil']['native_time_dt'],merged_dict['arm_ceil']['native_cbh_2']*1.e-3,s=2,c='magenta',label='CBH2')
if np.all(merged_dict['arm_ceil']['native_cbh_3'] != np.nan):
    ax2.scatter(merged_dict['arm_ceil']['native_time_dt'],merged_dict['arm_ceil']['native_cbh_3']*1.e-3,s=2,c='cyan',label='CBH3')
ax2.legend(fontsize=Fontsize,loc='upper left',ncol=1)
        
# Plot Interpolated Attenuated Backscatter
cmap = matplotlib.cm.get_cmap("jet").copy()
cmap.set_under('navy')
cmap.set_bad('grey')
dumid = np.where(merged_dict['basta']['height'] <= dum_y_lim*1.e3)
dumid = np.squeeze(dumid)
ceil_backscatter_interp_plot = ax3.pcolormesh(merged_dict['basta']['time_dt'],\
                                     merged_dict['arm_ceil']['interp_height'][dumid]*1.e-3,\
                                     merged_dict['arm_ceil']['backscatter'][dumid,:],\
                                     cmap=cmap,
                                     vmin=-8,vmax=-3)
dum_ticks = [-8,-7,-6,-5,-4,-3]
ceil_backscatter_interp_cbar = fig.colorbar(ceil_backscatter_interp_plot,ax=ax3,pad=0.005,ticks=dum_ticks)
ceil_backscatter_interp_cbar.ax.set_ylabel('$log_{10}$($\\beta_{att}$) [sr$^{-1}$ m$^{-1}$]',fontsize=Fontsize)
ceil_backscatter_interp_cbar.ax.tick_params(labelsize=Fontsize)

if np.all(merged_dict['arm_ceil']['cbh_1'] != np.nan):
    ax3.scatter(merged_dict['basta']['time_dt'],merged_dict['arm_ceil']['cbh_1']*1.e-3,s=2,c='k',label='CBH1')
if np.all(merged_dict['arm_ceil']['cbh_2'] != np.nan):
    ax3.scatter(merged_dict['basta']['time_dt'],merged_dict['arm_ceil']['cbh_2']*1.e-3,s=2,c='magenta',label='CBH2')
if np.all(merged_dict['arm_ceil']['cbh_3'] != np.nan):
    ax3.scatter(merged_dict['basta']['time_dt'],merged_dict['arm_ceil']['cbh_3']*1.e-3,s=2,c='cyan',label='CBH3')
ax3.legend(fontsize=Fontsize,loc='upper left',ncol=1)



for ax in axlist:
    ax.set_ylabel('Height [km]',fontsize=Fontsize)
    ax.set_xlabel('UTC Time [HH:MM]',fontsize=Fontsize)
    ax.tick_params(labelsize=Fontsize)
    ax.xaxis.set_major_formatter(dfmt)
    ax.grid(True)
    ax.set_ylim(0,1)
    
ax1.set_title('BASTA Reflectivity',fontsize=Fontsize*1.25,fontweight='bold')
dumstr = '$log_{10}$($\\beta_{att}$)'
ax2.set_title('ARM CEIL Native {}'.format(dumstr),fontsize=Fontsize*1.25,fontweight='bold')
ax3.set_title('ARM CEIL Interpolated {}'.format(dumstr),fontsize=Fontsize*1.25,fontweight='bold')
    
datestr = datetime.datetime(basta_time_dt[0].year,basta_time_dt[0].month,basta_time_dt[0].day).strftime("%m/%d/%Y")
plt.suptitle(datestr,fontsize=Fontsize*2)
plt.subplots_adjust(hspace=0.4)
plt.show()
plt.close()

In [ ]:
#----------------------------------------------------
# Plot to check interpolation of Univ. of Canterbury
# CEIL beta_att to radar time-height grid
#----------------------------------------------------
fig = plt.figure(figsize=(14,12))
ax1=fig.add_subplot(311)
ax2=fig.add_subplot(312)
ax3=fig.add_subplot(313)
axlist = [ax1,ax2,ax3]
Fontsize=14
dfmt = mdates.DateFormatter('%H:%M')

dum_y_lim = 1.
# Plot Reflectivity
dum_ref = merged_dict['basta']['ref'].copy()
dum_ref[(dum_ref > -999.) & (dum_ref < -50.)] = -50.
dum_ref[dum_ref > 20.] = 20.
cmap = matplotlib.cm.get_cmap("nipy_spectral").copy()
cmap.set_under('w')
cmap.set_bad('grey')
dumid = np.where(merged_dict['basta']['height'] <= dum_y_lim*1.e3)
dumid = np.squeeze(dumid)

basta_bin_edges = np.arange(0,np.max(merged_dict['basta']['height'])+12.5+25.,25.)
ref_plot = ax1.pcolormesh(merged_dict['basta']['time_dt'],\
                            merged_dict['basta']['height'][dumid]/1.e3,\
                            dum_ref[dumid,:],\
                            cmap=cmap,\
                            vmin=-50.1,vmax=20.1)
ref_cbar = fig.colorbar(ref_plot,ax=ax1,pad=0.005)
ref_cbar.ax.set_ylabel('Reflectivity [dBZ]',fontsize=Fontsize)
ref_cbar.ax.tick_params(labelsize=Fontsize)
if np.all(merged_dict['aad_ceil']['cbh_1'] != np.nan):
    ax1.scatter(merged_dict['basta']['time_dt'],merged_dict['aad_ceil']['cbh_1']*1.e-3,s=2,c='k',label='CBH1')
if np.all(merged_dict['aad_ceil']['cbh_2'] != np.nan):
    ax1.scatter(merged_dict['basta']['time_dt'],merged_dict['aad_ceil']['cbh_2']*1.e-3,s=2,c='magenta',label='CBH2')
if np.all(merged_dict['aad_ceil']['cbh_3'] != np.nan):
    ax1.scatter(merged_dict['basta']['time_dt'],merged_dict['aad_ceil']['cbh_3']*1.e-3,s=2,c='cyan',label='CBH3')
ax1.legend(fontsize=Fontsize,loc='upper left',ncol=1)


# Plot Attenuated Backscatter
dum_aad_ceil_backscatter = merged_dict['aad_ceil']['native_backscatter'].copy()
cmap = matplotlib.cm.get_cmap("jet").copy()
cmap.set_under('navy')
cmap.set_bad('grey')
dum_aad_ceil_backscatter[dum_aad_ceil_backscatter < -8.] = -8.
dum_aad_ceil_backscatter[dum_aad_ceil_backscatter > -3.] = -3.
dumid = np.where(merged_dict['aad_ceil']['native_range'] <= dum_y_lim*1.e3)
dumid = np.squeeze(dumid)
ceil_backscatter_plot = ax2.pcolormesh(merged_dict['aad_ceil']['native_time_dt'],\
                                     merged_dict['aad_ceil']['native_range'][dumid]*1.e-3,\
                                     np.transpose(dum_aad_ceil_backscatter[:,dumid]),\
                                     cmap=cmap,
                                     vmin=-8,vmax=-3)
dum_ticks = [-8,-7,-6,-5,-4,-3]

ceil_backscatter_cbar = fig.colorbar(ceil_backscatter_plot,ax=ax2,pad=0.005,ticks=dum_ticks)
ceil_backscatter_cbar.ax.set_ylabel('$log_{10}$($\\beta_{att}$) [sr$^{-1}$ m$^{-1}$]',fontsize=Fontsize)
ceil_backscatter_cbar.ax.tick_params(labelsize=Fontsize)
if np.all(merged_dict['aad_ceil']['native_cbh_1'] != np.nan):
    ax2.scatter(merged_dict['aad_ceil']['native_time_dt'],merged_dict['aad_ceil']['native_cbh_1']*1.e-3,s=2,c='k',label='CBH1')
if np.all(merged_dict['aad_ceil']['native_cbh_2'] != np.nan):
    ax2.scatter(merged_dict['aad_ceil']['native_time_dt'],merged_dict['aad_ceil']['native_cbh_2']*1.e-3,s=2,c='magenta',label='CBH2')
if np.all(merged_dict['aad_ceil']['native_cbh_3'] != np.nan):
    ax2.scatter(merged_dict['aad_ceil']['native_time_dt'],merged_dict['aad_ceil']['native_cbh_3']*1.e-3,s=2,c='cyan',label='CBH3')
ax2.legend(fontsize=Fontsize,loc='upper left',ncol=1)
        
# Plot Interpolated Attenuated Backscatter
cmap = matplotlib.cm.get_cmap("jet").copy()
cmap.set_under('navy')
cmap.set_bad('grey')
dumid = np.where(merged_dict['basta']['height'] <= dum_y_lim*1.e3)
dumid = np.squeeze(dumid)

ceil_backscatter_interp_plot = ax3.pcolormesh(merged_dict['basta']['time_dt'],\
                                     merged_dict['aad_ceil']['interp_height'][dumid]*1.e-3,\
                                     merged_dict['aad_ceil']['backscatter'][dumid,:],\
                                     cmap=cmap,
                                     vmin=-8,vmax=-3)
dum_ticks = [-8,-7,-6,-5,-4,-3]
ceil_backscatter_interp_cbar = fig.colorbar(ceil_backscatter_interp_plot,ax=ax3,pad=0.005,ticks=dum_ticks)
ceil_backscatter_interp_cbar.ax.set_ylabel('$log_{10}$($\\beta_{att}$) [sr$^{-1}$ m$^{-1}$]',fontsize=Fontsize)
ceil_backscatter_interp_cbar.ax.tick_params(labelsize=Fontsize)

if np.all(merged_dict['aad_ceil']['cbh_1'] != np.nan):
    ax3.scatter(merged_dict['basta']['time_dt'],merged_dict['aad_ceil']['cbh_1']*1.e-3,s=2,c='k',label='CBH1')
if np.all(merged_dict['aad_ceil']['cbh_2'] != np.nan):
    ax3.scatter(merged_dict['basta']['time_dt'],merged_dict['aad_ceil']['cbh_2']*1.e-3,s=2,c='magenta',label='CBH2')
if np.all(merged_dict['aad_ceil']['cbh_3'] != np.nan):
    ax3.scatter(merged_dict['basta']['time_dt'],merged_dict['aad_ceil']['cbh_3']*1.e-3,s=2,c='cyan',label='CBH3')
ax3.legend(fontsize=Fontsize,loc='upper left',ncol=1)



for ax in axlist:
    ax.set_ylabel('Height [km]',fontsize=Fontsize)
    ax.set_xlabel('UTC Time [HH:MM]',fontsize=Fontsize)
    ax.tick_params(labelsize=Fontsize)
    ax.xaxis.set_major_formatter(dfmt)
    ax.grid(True)
    ax.set_ylim(0,1)
    
ax1.set_title('BASTA Reflectivity',fontsize=Fontsize*1.25,fontweight='bold')
dumstr = '$log_{10}$($\\beta_{att}$)'
ax2.set_title('Univ. of Canterbury CEIL Native {}'.format(dumstr),fontsize=Fontsize*1.25,fontweight='bold')
ax3.set_title('Univ. of Canterbury CEIL Interpolated {}'.format(dumstr),fontsize=Fontsize*1.25,fontweight='bold')
    
datestr = datetime.datetime(basta_time_dt[0].year,basta_time_dt[0].month,basta_time_dt[0].day).strftime("%m/%d/%Y")
plt.suptitle(datestr,fontsize=Fontsize*2)
plt.subplots_adjust(hspace=0.4)
plt.show()
plt.close()